# Vision-Language Models for Scene Understanding and VQA
## Master-Level Research Project: BLIP-2 + Scene Reasoning Module

**Author:** Research Implementation  
**Target Environment:** Google Colab (Free/Pro with GPU)

This notebook implements a complete research project for Visual Question Answering using:
1. **Baseline:** BLIP-2 pretrained model (Salesforce/blip2-opt-2.7b)
2. **Proposed:** BLIP-2 + Custom Scene Reasoning Module with spatial/relational attention

### Features:
- Complete modular codebase under `/content/VLM_Thesis`
- VQAv2 dataset integration via HuggingFace
- Ablation study configurations
- Academic reporting outputs
- Smoke test mode for quick verification

---

## Section 1: Environment Setup and Dependencies

Install all required packages for the VLM research project. This cell handles:
- PyTorch and Transformers ecosystem
- Accelerate for distributed training
- TensorBoard for logging
- Additional utilities

In [ ]:
# ============================================================================
# 🚀 COLAB SETUP - RUN THIS CELL FIRST!
# ============================================================================
"""
Vision-Language Model Research Project
BLIP-2 + Scene Reasoning Module for VQA

This notebook is optimized for Google Colab with GPU runtime.
Run cells in order from top to bottom.
"""

import os
import sys
import subprocess
from pathlib import Path

# ============================================================================
# 1. ENVIRONMENT DETECTION
# ============================================================================
def detect_environment():
    """Detect execution environment."""
    try:
        import google.colab
        return "colab"
    except ImportError:
        if sys.platform == "darwin":
            return "mac"
        return "local"

ENV = detect_environment()
print(f"🖥️  Environment: {ENV.upper()}")

# ============================================================================
# 2. GPU CHECK (Colab)
# ============================================================================
if ENV == "colab":
    import torch
    if not torch.cuda.is_available():
        print("⚠️  WARNING: No GPU detected!")
        print("   Go to: Runtime → Change runtime type → GPU")
        print("   Then restart and run this cell again.")
    else:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"✅ GPU: {gpu_name} ({gpu_mem:.1f} GB)")

# ============================================================================
# 3. PROJECT CONFIGURATION
# ============================================================================
PROJECT_NAME = "VLM_Thesis"

if ENV == "colab":
    PROJECT_ROOT = f"/content/{PROJECT_NAME}"
    MOUNT_DRIVE = True  # Set to False to disable Drive sync
else:
    PROJECT_ROOT = os.getcwd()
    MOUNT_DRIVE = False

# ============================================================================
# 4. GOOGLE DRIVE MOUNT (Optional)
# ============================================================================
DRIVE_OUTPUT_DIR = None
if MOUNT_DRIVE and ENV == "colab":
    try:
        from google.colab import drive
        drive.mount('/content/drive', force_remount=False)
        DRIVE_OUTPUT_DIR = f"/content/drive/MyDrive/{PROJECT_NAME}_Outputs"
        os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
        print(f"✅ Drive mounted → {DRIVE_OUTPUT_DIR}")
    except Exception as e:
        print(f"⚠️  Drive mount skipped: {e}")

# ============================================================================
# 5. CLONE/UPDATE FROM GITHUB
# ============================================================================
REPO_URL = "https://github.com/Saif-Amer-20/Vision-Language.git"

if ENV == "colab":
    if os.path.exists(PROJECT_ROOT):
        # Pull latest changes
        os.chdir(PROJECT_ROOT)
        result = subprocess.run(["git", "pull"], capture_output=True, text=True)
        print(f"📥 Git pull: {result.stdout.strip() or result.stderr.strip()}")
    else:
        # Clone repository
        subprocess.run(["git", "clone", REPO_URL, PROJECT_ROOT], check=True)
        print(f"📥 Cloned repository to {PROJECT_ROOT}")
        os.chdir(PROJECT_ROOT)
else:
    print(f"📂 Using local project: {PROJECT_ROOT}")

# ============================================================================
# 6. INSTALL DEPENDENCIES
# ============================================================================
print("\n📦 Installing dependencies...")
packages = [
    "torch>=2.0.0",
    "torchvision>=0.15.0",
    "transformers>=4.35.0",
    "accelerate>=0.24.0",
    "datasets>=2.14.0",
    "Pillow>=9.0.0",
    "pyyaml>=6.0",
    "tensorboard>=2.14.0",
    "tqdm>=4.65.0",
    "pandas>=2.0.0",
    "numpy>=1.24.0",
    "scikit-learn>=1.3.0",
    "matplotlib>=3.7.0",
    "seaborn>=0.12.0",
]

for pkg in packages:
    subprocess.run([sys.executable, "-m", "pip", "install", "-q", pkg], 
                   capture_output=True)
print("✅ Dependencies installed")

# ============================================================================
# 7. VERIFY SETUP
# ============================================================================
import torch
import transformers

print(f"\n{'='*50}")
print(f"📊 SETUP COMPLETE")
print(f"{'='*50}")
print(f"   Environment:    {ENV}")
print(f"   Project Root:   {PROJECT_ROOT}")
print(f"   PyTorch:        {torch.__version__}")
print(f"   Transformers:   {transformers.__version__}")
print(f"   CUDA:           {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU:            {torch.cuda.get_device_name(0)}")
print(f"   Drive Sync:     {DRIVE_OUTPUT_DIR or 'Disabled'}")
print(f"{'='*50}")

In [ ]:
# ============================================================================
# ⚙️ RUNTIME CONFIGURATION
# ============================================================================
"""
Configure runtime settings based on environment.
This cell sets up paths and caching for optimal performance.
"""
import os
import sys
from pathlib import Path

# Use PROJECT_ROOT from setup cell
try:
    PROJECT_ROOT
except NameError:
    PROJECT_ROOT = "/content/VLM_Thesis"

# Add src to Python path for imports
src_path = str(Path(PROJECT_ROOT) / "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

# HuggingFace cache configuration (use local disk for speed)
os.environ["HF_HOME"] = "/root/.cache/huggingface"
os.environ["HF_DATASETS_CACHE"] = "/root/.cache/huggingface/datasets"
os.environ["TRANSFORMERS_CACHE"] = "/root/.cache/huggingface/hub"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Disable wandb by default (use TensorBoard instead)
os.environ["WANDB_DISABLED"] = "true"

# Memory optimization for Colab
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:512"

print(f"✅ Runtime configured")
print(f"   Project: {PROJECT_ROOT}")
print(f"   Python path updated for local imports")

## Section 2: Project Structure Creation

Create the complete folder structure for the research project. This follows a clean, modular architecture suitable for thesis-grade code.

In [ ]:
# ============================================================================
# 📂 CREATE PROJECT STRUCTURE
# ============================================================================
"""
Creates the complete project structure with all necessary directories
and __init__.py files for Python packages.
"""
import os
from pathlib import Path

# Use PROJECT_ROOT from previous cell (or set default)
try:
    PROJECT_ROOT
except NameError:
    PROJECT_ROOT = "/content/VLM_Thesis"

print(f"📂 Project Root: {PROJECT_ROOT}")

# Ensure we're in project directory
os.makedirs(PROJECT_ROOT, exist_ok=True)
os.chdir(PROJECT_ROOT)

# ============================================================================
# DIRECTORY STRUCTURE
# ============================================================================
directories = [
    "configs",
    "data",
    "src",
    "src/data",
    "src/models", 
    "src/training",
    "src/evaluation",
    "src/utils",
    "scripts",
    "outputs",
    "outputs/checkpoints",
    "outputs/logs",
    "outputs/results",
    "thesis_assets",
    "docs",
]

for dir_path in directories:
    full_path = Path(PROJECT_ROOT) / dir_path
    full_path.mkdir(parents=True, exist_ok=True)

# ============================================================================
# CREATE __init__.py FILES
# ============================================================================
init_packages = [
    "src",
    "src/data",
    "src/models",
    "src/training",
    "src/evaluation",
    "src/utils",
]

for pkg in init_packages:
    init_path = Path(PROJECT_ROOT) / pkg / "__init__.py"
    if not init_path.exists():
        init_path.write_text('"""Package initialization."""\n')

print("✅ Project structure ready!")

# ============================================================================
# DISPLAY STRUCTURE
# ============================================================================
def show_tree(path, prefix="", max_depth=2, current_depth=0):
    """Display directory tree."""
    if current_depth >= max_depth:
        return
    path = Path(path)
    items = sorted([p for p in path.iterdir() if not p.name.startswith('.')])
    for i, item in enumerate(items):
        is_last = i == len(items) - 1
        connector = "└── " if is_last else "├── "
        print(f"{prefix}{connector}{item.name}{'/' if item.is_dir() else ''}")
        if item.is_dir():
            extension = "    " if is_last else "│   "
            show_tree(item, prefix + extension, max_depth, current_depth + 1)

print(f"\n📂 {Path(PROJECT_ROOT).name}/")
show_tree(PROJECT_ROOT)

# ============================================================================
# 📝 FILE WRITER UTILITY
# ============================================================================
def write_file(relative_path: str, content: str):
    """
    Write content to a file in the project directory.
    
    Args:
        relative_path: Path relative to PROJECT_ROOT (e.g., 'src/models/blip2.py')
        content: File content to write
    """
    file_path = Path(PROJECT_ROOT) / relative_path
    file_path.parent.mkdir(parents=True, exist_ok=True)
    file_path.write_text(content)
    print(f"✅ Created: {relative_path}")

# Make PROJECT_ROOT available globally for all cells
import builtins
builtins.PROJECT_ROOT = PROJECT_ROOT
builtins.write_file = write_file
print(f"\n🔧 write_file() helper ready - use: write_file('src/file.py', content)")

## Section 3: Configuration System Implementation

Implement a robust YAML-based configuration system with:
- Nested configuration support
- CLI override capabilities
- Type-safe dataclass objects
- Colab-safe default values

In [ ]:
# ============================================================================
# FILE: src/utils/config.py
# Configuration system with YAML support, CLI overrides, and execution profiles
# ============================================================================

config_py_content = '''"""
Configuration system for VLM research project.

Provides YAML-based configuration with CLI overrides, type-safe dataclasses,
execution profiles (colab_train, mac_dev, eval_only), and validation.

Execution Profiles:
    - colab_train: Full training on Colab GPU (default)
    - mac_dev: Local development on Mac (smoke/sanity only, no long training)
    - eval_only: Evaluation mode (no training allowed)
"""

from dataclasses import dataclass, field, asdict
from typing import Optional, List, Dict, Any, Literal
from enum import Enum
import yaml
import os
import sys
import platform
import argparse
from pathlib import Path


class ExecutionProfile(Enum):
    """Execution profile for different environments."""
    COLAB_TRAIN = "colab_train"  # Full training on Colab GPU
    MAC_DEV = "mac_dev"          # Local development (smoke tests only)
    EVAL_ONLY = "eval_only"      # Evaluation only (no training)
    
    @classmethod
    def from_string(cls, s: str) -> 'ExecutionProfile':
        """Create from string value."""
        mapping = {
            'colab_train': cls.COLAB_TRAIN,
            'mac_dev': cls.MAC_DEV,
            'eval_only': cls.EVAL_ONLY,
        }
        if s.lower() not in mapping:
            raise ValueError(f"Unknown execution profile: {s}. Valid: {list(mapping.keys())}")
        return mapping[s.lower()]


@dataclass
class RuntimeConfig:
    """Runtime configuration for execution environment."""
    execution_profile: str = "colab_train"  # colab_train, mac_dev, eval_only
    sync_to_drive: bool = False             # Sync outputs to Google Drive
    drive_mount_path: str = "/content/drive/MyDrive/VLM_Thesis_Outputs"
    
    # Safety limits for mac_dev profile
    mac_dev_max_steps: int = 10             # Max training steps allowed on Mac
    mac_dev_max_samples: int = 50           # Max samples allowed on Mac
    mac_dev_allow_checkpoints: bool = False # Disable checkpoint saving on Mac
    
    def get_profile(self) -> ExecutionProfile:
        """Get execution profile as enum."""
        return ExecutionProfile.from_string(self.execution_profile)
    
    def is_training_allowed(self) -> bool:
        """Check if training is allowed in current profile."""
        return self.get_profile() != ExecutionProfile.EVAL_ONLY
    
    def is_full_training_allowed(self) -> bool:
        """Check if full (non-smoke) training is allowed."""
        return self.get_profile() == ExecutionProfile.COLAB_TRAIN


@dataclass
class DataConfig:
    """Dataset configuration."""
    dataset_name: str = "HuggingFaceM4/VQAv2"
    split_train: str = "train"
    split_val: str = "validation"
    max_samples_train: Optional[int] = None  # None = use all
    max_samples_val: Optional[int] = None
    image_size: int = 224
    max_question_length: int = 32
    max_answer_length: int = 16
    num_workers: int = 2
    cache_dir: str = "/root/.cache/huggingface/datasets"
    
    
@dataclass
class ModelConfig:
    """Model configuration."""
    model_name: str = "Salesforce/blip2-opt-2.7b"
    use_scene_reasoning: bool = False
    freeze_vision_encoder: bool = True
    freeze_llm: bool = True
    freeze_qformer: bool = False
    
    # Scene Reasoning Module settings
    scene_reasoning_dim: int = 768
    scene_reasoning_heads: int = 8
    scene_reasoning_layers: int = 2
    use_spatial_encoding: bool = True
    use_relation_attention: bool = True
    spatial_encoding_dim: int = 64
    
    # Generation settings
    max_new_tokens: int = 16
    num_beams: int = 3
    
    
@dataclass
class TrainingConfig:
    """Training configuration."""
    batch_size: int = 1  # Safe for Colab Free
    gradient_accumulation_steps: int = 8
    learning_rate: float = 1e-5
    weight_decay: float = 0.01
    num_epochs: int = 3
    max_steps: Optional[int] = None  # If set, overrides num_epochs
    warmup_ratio: float = 0.1
    lr_scheduler_type: str = "cosine"
    
    # Precision and memory
    fp16: bool = True
    gradient_checkpointing: bool = False
    max_grad_norm: float = 1.0
    
    # Device override (auto, cpu, cuda, mps)
    device: str = "auto"
    
    # Checkpointing
    save_strategy: str = "epoch"  # "epoch" or "steps"
    save_steps: int = 500
    save_total_limit: int = 2
    save_checkpoints: bool = True  # Can disable for dev runs
    eval_steps: int = 500
    
    # Early stopping
    early_stopping: bool = False
    early_stopping_patience: int = 3
    
    # Smoke test mode
    smoke_test: bool = False
    smoke_test_samples: int = 32
    smoke_test_steps: int = 5
    
    
@dataclass
class LoggingConfig:
    """Logging configuration."""
    output_dir: str = "/content/VLM_Thesis/outputs"
    experiment_name: str = "vqa_experiment"
    use_tensorboard: bool = True
    use_wandb: bool = False
    wandb_project: str = "vlm-vqa-research"
    log_every_n_steps: int = 10
    
    
@dataclass
class Config:
    """Main configuration combining all sub-configs."""
    data: DataConfig = field(default_factory=DataConfig)
    model: ModelConfig = field(default_factory=ModelConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    logging: LoggingConfig = field(default_factory=LoggingConfig)
    runtime: RuntimeConfig = field(default_factory=RuntimeConfig)
    seed: int = 42
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert config to dictionary."""
        return asdict(self)
    
    def save(self, path: str) -> None:
        """Save configuration to YAML file."""
        os.makedirs(os.path.dirname(path), exist_ok=True)
        with open(path, 'w') as f:
            yaml.dump(self.to_dict(), f, default_flow_style=False, sort_keys=False)
    
    @classmethod
    def from_yaml(cls, path: str) -> 'Config':
        """Load configuration from YAML file."""
        with open(path, 'r') as f:
            config_dict = yaml.safe_load(f)
        return cls.from_dict(config_dict)
    
    @classmethod
    def from_dict(cls, config_dict: Dict[str, Any]) -> 'Config':
        """Create config from dictionary."""
        data_config = DataConfig(**config_dict.get('data', {}))
        model_config = ModelConfig(**config_dict.get('model', {}))
        training_config = TrainingConfig(**config_dict.get('training', {}))
        logging_config = LoggingConfig(**config_dict.get('logging', {}))
        runtime_config = RuntimeConfig(**config_dict.get('runtime', {}))
        
        return cls(
            data=data_config,
            model=model_config,
            training=training_config,
            logging=logging_config,
            runtime=runtime_config,
            seed=config_dict.get('seed', 42)
        )
    
    def apply_cli_overrides(self, args: argparse.Namespace) -> 'Config':
        """Apply CLI argument overrides to config."""
        # Handle execution profile override
        if hasattr(args, 'execution_profile') and args.execution_profile:
            self.runtime.execution_profile = args.execution_profile
            
        # Handle Drive sync
        if hasattr(args, 'sync_to_drive') and args.sync_to_drive:
            self.runtime.sync_to_drive = True
        
        # Handle common CLI overrides
        if hasattr(args, 'smoke_test') and args.smoke_test:
            self.training.smoke_test = True
            self.training.max_steps = self.training.smoke_test_steps
            self.data.max_samples_train = self.training.smoke_test_samples
            self.data.max_samples_val = self.training.smoke_test_samples // 2
            
        if hasattr(args, 'batch_size') and args.batch_size:
            self.training.batch_size = args.batch_size
            
        if hasattr(args, 'lr') and args.lr:
            self.training.learning_rate = args.lr
            
        if hasattr(args, 'epochs') and args.epochs:
            self.training.num_epochs = args.epochs
            
        if hasattr(args, 'output_dir') and args.output_dir:
            self.logging.output_dir = args.output_dir
            
        if hasattr(args, 'experiment_name') and args.experiment_name:
            self.logging.experiment_name = args.experiment_name
        
        # Apply execution profile constraints
        self._apply_profile_constraints()
            
        return self
    
    def _apply_profile_constraints(self) -> None:
        """Apply constraints based on execution profile."""
        profile = self.runtime.get_profile()
        
        if profile == ExecutionProfile.MAC_DEV:
            # Safety guard: Force smoke-test-like limits on Mac
            if not self.training.smoke_test:
                if self.training.max_steps is None or self.training.max_steps > self.runtime.mac_dev_max_steps:
                    print(f"⚠️  mac_dev profile: Limiting max_steps to {self.runtime.mac_dev_max_steps}")
                    self.training.max_steps = self.runtime.mac_dev_max_steps
                    
                if self.data.max_samples_train is None or self.data.max_samples_train > self.runtime.mac_dev_max_samples:
                    print(f"⚠️  mac_dev profile: Limiting train samples to {self.runtime.mac_dev_max_samples}")
                    self.data.max_samples_train = self.runtime.mac_dev_max_samples
                    self.data.max_samples_val = min(self.runtime.mac_dev_max_samples // 2, 25)
            
            # Disable checkpoints on Mac by default
            if not self.runtime.mac_dev_allow_checkpoints:
                self.training.save_checkpoints = False
                
            # Force CPU/MPS on Mac
            if self.training.device == "auto":
                self.training.device = "mps" if platform.system() == "Darwin" else "cpu"
            
            # Disable fp16 on Mac (MPS has limited support)
            if self.training.fp16:
                print("⚠️  mac_dev profile: Disabling fp16 (not fully supported on MPS)")
                self.training.fp16 = False
                
        elif profile == ExecutionProfile.EVAL_ONLY:
            # No training in eval mode
            self.training.num_epochs = 0
            self.training.max_steps = 0
    
    def validate(self) -> None:
        """Validate configuration values."""
        assert self.training.batch_size >= 1, "Batch size must be >= 1"
        assert self.training.gradient_accumulation_steps >= 1, "Gradient accumulation must be >= 1"
        assert self.training.learning_rate > 0, "Learning rate must be > 0"
        assert self.model.scene_reasoning_dim > 0, "Scene reasoning dim must be > 0"
        
        # Validate execution profile
        profile = self.runtime.get_profile()
        
        # Warn about memory-intensive settings
        if self.training.batch_size > 2 and not self.training.fp16:
            print("⚠️ Warning: batch_size > 2 without fp16 may cause OOM on Colab Free")
        
        if not self.model.freeze_vision_encoder or not self.model.freeze_llm:
            print("⚠️ Warning: Unfreezing backbone may cause OOM. Consider gradient checkpointing.")
        
        # Profile-specific validation
        if profile == ExecutionProfile.MAC_DEV:
            self._validate_mac_dev_safety()
    
    def _validate_mac_dev_safety(self) -> None:
        """Validate safety constraints for mac_dev profile."""
        if self.training.max_steps is None or self.training.max_steps > self.runtime.mac_dev_max_steps:
            raise ValueError(
                f"🛑 SAFETY GUARD: mac_dev profile does not allow training with "
                f"max_steps > {self.runtime.mac_dev_max_steps}. "
                f"Use --execution_profile colab_train for full training."
            )
        
        if self.data.max_samples_train is None or self.data.max_samples_train > self.runtime.mac_dev_max_samples:
            raise ValueError(
                f"🛑 SAFETY GUARD: mac_dev profile does not allow training with "
                f"max_samples > {self.runtime.mac_dev_max_samples}. "
                f"Use --execution_profile colab_train for full training."
            )
        
        print(f"✅ mac_dev safety check passed (max_steps={self.training.max_steps}, "
              f"max_samples={self.data.max_samples_train})")
    
    def get_effective_device(self) -> str:
        """Get effective device based on config and availability."""
        import torch
        
        if self.training.device != "auto":
            return self.training.device
        
        if torch.cuda.is_available():
            return "cuda"
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            return "mps"
        else:
            return "cpu"
    
    def print_profile_info(self) -> None:
        """Print execution profile information."""
        profile = self.runtime.get_profile()
        
        print(f"\\n{'='*60}")
        print(f"⚙️  EXECUTION PROFILE: {profile.value}")
        print(f"{'='*60}")
        
        if profile == ExecutionProfile.COLAB_TRAIN:
            print("   Mode: Full Training on Colab GPU")
            print("   - All training features enabled")
            print("   - Checkpoints saved to /content/VLM_Thesis/outputs")
            if self.runtime.sync_to_drive:
                print(f"   - Drive sync enabled: {self.runtime.drive_mount_path}")
        elif profile == ExecutionProfile.MAC_DEV:
            print("   Mode: Local Development (Mac)")
            print(f"   - Max steps: {self.runtime.mac_dev_max_steps}")
            print(f"   - Max samples: {self.runtime.mac_dev_max_samples}")
            print(f"   - Device: {self.training.device}")
            print(f"   - Checkpoints: {'enabled' if self.training.save_checkpoints else 'disabled'}")
            print("   - ⚠️  For full training, use Colab!")
        elif profile == ExecutionProfile.EVAL_ONLY:
            print("   Mode: Evaluation Only")
            print("   - Training disabled")
            print("   - Use for inference and evaluation only")
        
        print(f"{'='*60}\\n")


def get_argument_parser() -> argparse.ArgumentParser:
    """Create argument parser for CLI."""
    parser = argparse.ArgumentParser(description="VLM VQA Training")
    
    # Config file
    parser.add_argument("--config", type=str, required=True, help="Path to config YAML")
    
    # Execution profile (NEW)
    parser.add_argument("--execution_profile", type=str, default=None,
                        choices=['colab_train', 'mac_dev', 'eval_only'],
                        help="Execution profile: colab_train (default), mac_dev, eval_only")
    parser.add_argument("--sync_to_drive", action='store_true',
                        help="Sync outputs to Google Drive (Colab only)")
    
    # Common overrides
    parser.add_argument("--smoke_test", type=lambda x: x.lower() == 'true', default=False)
    parser.add_argument("--batch_size", type=int, default=None)
    parser.add_argument("--lr", type=float, default=None)
    parser.add_argument("--epochs", type=int, default=None)
    parser.add_argument("--output_dir", type=str, default=None)
    parser.add_argument("--experiment_name", type=str, default=None)
    parser.add_argument("--ckpt", type=str, default=None, help="Checkpoint path for evaluation")
    
    return parser


def load_config(config_path: str, args: Optional[argparse.Namespace] = None) -> Config:
    """Load config from YAML and apply CLI overrides."""
    config = Config.from_yaml(config_path)
    
    if args is not None:
        config = config.apply_cli_overrides(args)
    
    config.validate()
    return config


def detect_environment() -> str:
    """Auto-detect execution environment."""
    # Check if running in Colab
    try:
        import google.colab
        return "colab_train"
    except ImportError:
        pass
    
    # Check if Mac
    if platform.system() == "Darwin":
        return "mac_dev"
    
    # Default to eval_only for unknown environments
    return "eval_only"
'''

# Write to file
config_path = "/content/VLM_Thesis/src/utils/config.py"
with open(config_path, 'w') as f:
    f.write(config_py_content)

print(f"✅ Created: {config_path}")

## Section 4: Utility Modules (Seed, IO, Logging)

Core utility modules for reproducibility, file I/O, and experiment logging.

In [ ]:
# ============================================================================
# FILE: src/utils/seed.py
# Deterministic reproducibility utilities
# ============================================================================

seed_py_content = '''"""
Seed utilities for reproducibility.

Ensures deterministic behavior across PyTorch, NumPy, Random, and CUDA operations.
"""

import random
import numpy as np
import torch
import os
from typing import Optional


def set_seed(seed: int = 42, deterministic: bool = True) -> None:
    """
    Set random seeds for reproducibility across all libraries.
    
    Args:
        seed: Random seed value
        deterministic: If True, use deterministic CUDA algorithms (may be slower)
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # For multi-GPU
        
    # Set environment variable for hash randomization
    os.environ['PYTHONHASHSEED'] = str(seed)
    
    if deterministic:
        # Deterministic algorithms (may impact performance)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False
        
        # For PyTorch >= 1.8
        if hasattr(torch, 'use_deterministic_algorithms'):
            try:
                torch.use_deterministic_algorithms(True, warn_only=True)
            except RuntimeError:
                pass  # Some operations don't have deterministic implementations
    else:
        # Enable cuDNN auto-tuner for better performance
        torch.backends.cudnn.deterministic = False
        torch.backends.cudnn.benchmark = True
    
    print(f"🎲 Random seed set to {seed} (deterministic={deterministic})")


def get_worker_init_fn(seed: int):
    """
    Get worker initialization function for DataLoader reproducibility.
    
    Args:
        seed: Base random seed
        
    Returns:
        Worker init function
    """
    def worker_init_fn(worker_id: int) -> None:
        worker_seed = seed + worker_id
        np.random.seed(worker_seed)
        random.seed(worker_seed)
        
    return worker_init_fn


def get_generator(seed: int) -> torch.Generator:
    """
    Get a seeded generator for DataLoader.
    
    Args:
        seed: Random seed
        
    Returns:
        Seeded PyTorch Generator
    """
    g = torch.Generator()
    g.manual_seed(seed)
    return g
'''

write_file("src/utils/seed.py", seed_py_content)

# ============================================================================
# FILE: src/utils/io.py
# Checkpoint saving/loading and file I/O utilities
# ============================================================================

io_py_content = '''"""
I/O utilities for checkpoints, JSON, and CSV handling.
"""

import torch
import json
import csv
import os
import shutil
from pathlib import Path
from typing import Dict, Any, Optional, List
from datetime import datetime


def save_checkpoint(
    state_dict: Dict[str, Any],
    path: str,
    is_best: bool = False,
    keep_last_n: int = 2
) -> None:
    """
    Save model checkpoint with optional best model tracking.
    
    Args:
        state_dict: Dictionary containing model state, optimizer state, etc.
        path: Path to save checkpoint
        is_best: Whether this is the best model so far
        keep_last_n: Number of recent checkpoints to keep
    """
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    # Add timestamp
    state_dict['timestamp'] = datetime.now().isoformat()
    
    # Save checkpoint
    torch.save(state_dict, path)
    print(f"💾 Checkpoint saved: {path}")
    
    # Save as best if applicable
    if is_best:
        best_path = path.replace('.pt', '_best.pt').replace('.pth', '_best.pth')
        if not best_path.endswith(('_best.pt', '_best.pth')):
            best_path = path.rsplit('.', 1)[0] + '_best.pt'
        shutil.copy(path, best_path)
        print(f"⭐ Best checkpoint saved: {best_path}")
    
    # Cleanup old checkpoints
    cleanup_old_checkpoints(os.path.dirname(path), keep_last_n)


def load_checkpoint(
    path: str,
    map_location: Optional[str] = None
) -> Dict[str, Any]:
    """
    Load model checkpoint.
    
    Args:
        path: Path to checkpoint file
        map_location: Device mapping for loading
        
    Returns:
        Loaded checkpoint dictionary
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Checkpoint not found: {path}")
    
    if map_location is None:
        map_location = 'cuda' if torch.cuda.is_available() else 'cpu'
    
    checkpoint = torch.load(path, map_location=map_location)
    print(f"📂 Checkpoint loaded: {path}")
    
    if 'timestamp' in checkpoint:
        print(f"   Saved at: {checkpoint['timestamp']}")
    if 'epoch' in checkpoint:
        print(f"   Epoch: {checkpoint['epoch']}")
    if 'global_step' in checkpoint:
        print(f"   Step: {checkpoint['global_step']}")
        
    return checkpoint


def cleanup_old_checkpoints(checkpoint_dir: str, keep_n: int = 2) -> None:
    """Remove old checkpoints, keeping only the most recent ones."""
    if not os.path.exists(checkpoint_dir):
        return
        
    checkpoints = []
    for f in os.listdir(checkpoint_dir):
        if f.endswith(('.pt', '.pth')) and 'best' not in f:
            path = os.path.join(checkpoint_dir, f)
            checkpoints.append((path, os.path.getmtime(path)))
    
    # Sort by modification time, oldest first
    checkpoints.sort(key=lambda x: x[1])
    
    # Remove oldest checkpoints
    while len(checkpoints) > keep_n:
        old_path, _ = checkpoints.pop(0)
        os.remove(old_path)
        print(f"🗑️ Removed old checkpoint: {old_path}")


def save_json(data: Any, path: str, indent: int = 2) -> None:
    """Save data to JSON file."""
    os.makedirs(os.path.dirname(path), exist_ok=True)
    with open(path, 'w') as f:
        json.dump(data, f, indent=indent, default=str)
    print(f"📄 JSON saved: {path}")


def load_json(path: str) -> Any:
    """Load data from JSON file."""
    with open(path, 'r') as f:
        return json.load(f)


def save_csv(
    data: List[Dict[str, Any]],
    path: str,
    fieldnames: Optional[List[str]] = None
) -> None:
    """Save list of dictionaries to CSV file."""
    if not data:
        print(f"⚠️ No data to save to {path}")
        return
        
    os.makedirs(os.path.dirname(path), exist_ok=True)
    
    if fieldnames is None:
        fieldnames = list(data[0].keys())
    
    with open(path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)
    
    print(f"📊 CSV saved: {path} ({len(data)} rows)")


def load_csv(path: str) -> List[Dict[str, Any]]:
    """Load CSV file to list of dictionaries."""
    with open(path, 'r') as f:
        reader = csv.DictReader(f)
        return list(reader)


def ensure_dir(path: str) -> str:
    """Ensure directory exists and return path."""
    os.makedirs(path, exist_ok=True)
    return path


def get_experiment_dir(base_dir: str, experiment_name: str) -> str:
    """
    Create experiment directory with timestamp.
    
    Returns path like: base_dir/experiment_name_20240101_120000/
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    exp_dir = os.path.join(base_dir, f"{experiment_name}_{timestamp}")
    os.makedirs(exp_dir, exist_ok=True)
    return exp_dir
'''

with open("/content/VLM_Thesis/src/utils/io.py", 'w') as f:
    f.write(io_py_content)
write_file("src/utils/io.py", io_py_content)

# ============================================================================
# FILE: src/utils/logging.py
# TensorBoard and optional W&B logging
# ============================================================================

logging_py_content = '''"""
Logging utilities for experiment tracking.

Supports TensorBoard (default) and optional Weights & Biases integration.
"""

import os
from typing import Dict, Any, Optional
from datetime import datetime
import torch


class ExperimentLogger:
    """
    Unified logger supporting TensorBoard and W&B.
    """
    
    def __init__(
        self,
        log_dir: str,
        experiment_name: str,
        use_tensorboard: bool = True,
        use_wandb: bool = False,
        wandb_project: Optional[str] = None,
        config: Optional[Dict[str, Any]] = None
    ):
        """
        Initialize experiment logger.
        
        Args:
            log_dir: Directory for logs
            experiment_name: Name of experiment
            use_tensorboard: Enable TensorBoard logging
            use_wandb: Enable Weights & Biases logging
            wandb_project: W&B project name
            config: Configuration to log
        """
        self.log_dir = log_dir
        self.experiment_name = experiment_name
        self.use_tensorboard = use_tensorboard
        self.use_wandb = use_wandb
        
        os.makedirs(log_dir, exist_ok=True)
        
        # Initialize TensorBoard
        self.tb_writer = None
        if use_tensorboard:
            from torch.utils.tensorboard import SummaryWriter
            tb_dir = os.path.join(log_dir, "tensorboard", experiment_name)
            self.tb_writer = SummaryWriter(log_dir=tb_dir)
            print(f"📊 TensorBoard logs: {tb_dir}")
        
        # Initialize W&B
        self.wandb_run = None
        if use_wandb:
            try:
                import wandb
                self.wandb_run = wandb.init(
                    project=wandb_project or "vlm-vqa",
                    name=experiment_name,
                    config=config,
                    dir=log_dir
                )
                print(f"📊 W&B run: {self.wandb_run.url}")
            except ImportError:
                print("⚠️ wandb not installed. Disabling W&B logging.")
                self.use_wandb = False
        
        self.step = 0
    
    def log_scalar(
        self,
        tag: str,
        value: float,
        step: Optional[int] = None
    ) -> None:
        """Log a scalar value."""
        step = step if step is not None else self.step
        
        if self.tb_writer:
            self.tb_writer.add_scalar(tag, value, step)
        
        if self.use_wandb and self.wandb_run:
            import wandb
            wandb.log({tag: value}, step=step)
    
    def log_scalars(
        self,
        main_tag: str,
        tag_scalar_dict: Dict[str, float],
        step: Optional[int] = None
    ) -> None:
        """Log multiple scalars under a main tag."""
        step = step if step is not None else self.step
        
        if self.tb_writer:
            self.tb_writer.add_scalars(main_tag, tag_scalar_dict, step)
        
        if self.use_wandb and self.wandb_run:
            import wandb
            logged = {f"{main_tag}/{k}": v for k, v in tag_scalar_dict.items()}
            wandb.log(logged, step=step)
    
    def log_metrics(
        self,
        metrics: Dict[str, float],
        step: Optional[int] = None,
        prefix: str = ""
    ) -> None:
        """Log a dictionary of metrics."""
        step = step if step is not None else self.step
        
        for key, value in metrics.items():
            tag = f"{prefix}/{key}" if prefix else key
            self.log_scalar(tag, value, step)
    
    def log_gpu_memory(self, step: Optional[int] = None) -> Optional[float]:
        """Log GPU memory usage if available."""
        if not torch.cuda.is_available():
            return None
        
        memory_gb = torch.cuda.max_memory_allocated() / 1e9
        self.log_scalar("system/gpu_memory_gb", memory_gb, step)
        return memory_gb
    
    def log_learning_rate(self, lr: float, step: Optional[int] = None) -> None:
        """Log learning rate."""
        self.log_scalar("train/learning_rate", lr, step)
    
    def log_text(self, tag: str, text: str, step: Optional[int] = None) -> None:
        """Log text data."""
        step = step if step is not None else self.step
        
        if self.tb_writer:
            self.tb_writer.add_text(tag, text, step)
    
    def log_histogram(
        self,
        tag: str,
        values: torch.Tensor,
        step: Optional[int] = None
    ) -> None:
        """Log histogram of values."""
        step = step if step is not None else self.step
        
        if self.tb_writer:
            self.tb_writer.add_histogram(tag, values, step)
    
    def log_image(
        self,
        tag: str,
        image: torch.Tensor,
        step: Optional[int] = None
    ) -> None:
        """Log an image (CHW format)."""
        step = step if step is not None else self.step
        
        if self.tb_writer:
            self.tb_writer.add_image(tag, image, step)
    
    def set_step(self, step: int) -> None:
        """Set current step."""
        self.step = step
    
    def close(self) -> None:
        """Close all loggers."""
        if self.tb_writer:
            self.tb_writer.close()
        
        if self.use_wandb and self.wandb_run:
            import wandb
            wandb.finish()
        
        print("📊 Loggers closed.")


def format_metrics(metrics: Dict[str, float], precision: int = 4) -> str:
    """Format metrics dictionary for printing."""
    parts = []
    for k, v in metrics.items():
        if isinstance(v, float):
            parts.append(f"{k}={v:.{precision}f}")
        else:
            parts.append(f"{k}={v}")
    return " | ".join(parts)


def get_gpu_memory_info() -> Dict[str, float]:
    """Get GPU memory information."""
    if not torch.cuda.is_available():
        return {}
    
    return {
        "allocated_gb": torch.cuda.memory_allocated() / 1e9,
        "reserved_gb": torch.cuda.memory_reserved() / 1e9,
        "max_allocated_gb": torch.cuda.max_memory_allocated() / 1e9,
    }
'''

write_file("src/utils/logging.py", logging_py_content)

## Section 5: Dataset Implementation (VQAv2 Loader)

Implement the VQA dataset loader with HuggingFace Datasets integration, BLIP-2 processor for image preprocessing, and configurable subset sampling for smoke tests.

In [ ]:
# ============================================================================
# FILE: src/data/vqa_dataset.py
# VQA Dataset implementation with HuggingFace integration
# ============================================================================

vqa_dataset_py_content = '''"""
VQA Dataset implementation for BLIP-2 training.

Supports VQAv2 dataset from HuggingFace with proper image preprocessing
and question tokenization using BLIP-2 processor.
"""

import torch
from torch.utils.data import Dataset, DataLoader
from typing import Dict, Any, Optional, List, Callable, Tuple
from PIL import Image
import io
from datasets import load_dataset
from transformers import Blip2Processor
import numpy as np


class VQADataset(Dataset):
    """
    VQA Dataset for BLIP-2 training.
    
    Loads VQAv2 from HuggingFace and processes images/questions using
    the BLIP-2 processor for model-ready inputs.
    """
    
    def __init__(
        self,
        processor: Blip2Processor,
        split: str = "train",
        dataset_name: str = "HuggingFaceM4/VQAv2",
        max_samples: Optional[int] = None,
        max_question_length: int = 32,
        max_answer_length: int = 16,
        cache_dir: Optional[str] = None,
        prompt_template: str = "Question: {question} Answer:"
    ):
        """
        Initialize VQA Dataset.
        
        Args:
            processor: BLIP-2 processor for image/text processing
            split: Dataset split (train, validation, test)
            dataset_name: HuggingFace dataset identifier
            max_samples: Maximum samples to load (None for all)
            max_question_length: Maximum question token length
            max_answer_length: Maximum answer token length
            cache_dir: Cache directory for dataset
            prompt_template: Template for question formatting
        """
        self.processor = processor
        self.split = split
        self.max_question_length = max_question_length
        self.max_answer_length = max_answer_length
        self.prompt_template = prompt_template
        
        print(f"📚 Loading VQA dataset: {dataset_name} ({split})...")
        
        # Load dataset from HuggingFace
        try:
            self.dataset = load_dataset(
                dataset_name,
                split=split,
                cache_dir=cache_dir,
                trust_remote_code=True
            )
        except Exception as e:
            print(f"⚠️ Error loading {dataset_name}: {e}")
            print("   Trying alternative dataset: Graphcore/vqa...")
            self.dataset = load_dataset(
                "Graphcore/vqa",
                split=split if split != "validation" else "validation",
                cache_dir=cache_dir,
                trust_remote_code=True
            )
        
        # Limit samples if specified
        if max_samples is not None and max_samples < len(self.dataset):
            self.dataset = self.dataset.select(range(max_samples))
        
        print(f"   Loaded {len(self.dataset)} samples")
        
        # Detect dataset column structure
        self._detect_columns()
    
    def _detect_columns(self) -> None:
        """Detect dataset column names for image, question, answer."""
        columns = self.dataset.column_names
        
        # Image column
        self.image_col = None
        for col in ['image', 'img', 'image_path', 'image_id']:
            if col in columns:
                self.image_col = col
                break
        
        # Question column
        self.question_col = None
        for col in ['question', 'text', 'query']:
            if col in columns:
                self.question_col = col
                break
        
        # Answer column (multiple possible names)
        self.answer_col = None
        for col in ['answer', 'answers', 'multiple_choice_answer', 'label']:
            if col in columns:
                self.answer_col = col
                break
        
        # Question ID column
        self.qid_col = None
        for col in ['question_id', 'id', 'idx']:
            if col in columns:
                self.qid_col = col
                break
        
        print(f"   Columns: image={self.image_col}, question={self.question_col}, answer={self.answer_col}")
    
    def _get_answer(self, item: Dict[str, Any]) -> str:
        """Extract answer string from dataset item."""
        answer = item.get(self.answer_col, "")
        
        # Handle different answer formats
        if isinstance(answer, list):
            # VQAv2 has list of answers - take most common or first
            if len(answer) > 0:
                if isinstance(answer[0], dict):
                    # Format: [{"answer": "yes", "answer_confidence": "yes"}, ...]
                    answers = [a.get("answer", "") for a in answer]
                else:
                    answers = answer
                # Return most common answer
                from collections import Counter
                answer = Counter(answers).most_common(1)[0][0]
            else:
                answer = ""
        elif isinstance(answer, dict):
            answer = answer.get("answer", str(answer))
        
        return str(answer)
    
    def _load_image(self, item: Dict[str, Any]) -> Image.Image:
        """Load and preprocess image from dataset item."""
        image_data = item.get(self.image_col)
        
        if image_data is None:
            # Create dummy image if missing
            return Image.new('RGB', (224, 224), color='gray')
        
        if isinstance(image_data, Image.Image):
            return image_data.convert('RGB')
        elif isinstance(image_data, bytes):
            return Image.open(io.BytesIO(image_data)).convert('RGB')
        elif isinstance(image_data, str):
            # Path to image
            return Image.open(image_data).convert('RGB')
        elif isinstance(image_data, dict) and 'bytes' in image_data:
            return Image.open(io.BytesIO(image_data['bytes'])).convert('RGB')
        else:
            # Try direct conversion
            return Image.fromarray(np.array(image_data)).convert('RGB')
    
    def __len__(self) -> int:
        return len(self.dataset)
    
    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """Get a single sample."""
        item = self.dataset[idx]
        
        # Load image
        image = self._load_image(item)
        
        # Get question and answer
        question = str(item.get(self.question_col, ""))
        answer = self._get_answer(item)
        
        # Format prompt
        prompt = self.prompt_template.format(question=question)
        
        # Process with BLIP-2 processor
        encoding = self.processor(
            images=image,
            text=prompt,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_question_length
        )
        
        # Tokenize answer for labels
        answer_encoding = self.processor.tokenizer(
            answer,
            return_tensors="pt",
            padding="max_length",
            truncation=True,
            max_length=self.max_answer_length
        )
        
        # Get question ID
        qid = item.get(self.qid_col, idx)
        
        return {
            "pixel_values": encoding["pixel_values"].squeeze(0),
            "input_ids": encoding["input_ids"].squeeze(0),
            "attention_mask": encoding["attention_mask"].squeeze(0),
            "labels": answer_encoding["input_ids"].squeeze(0),
            "question_id": qid,
            "question": question,
            "answer": answer,
        }


def vqa_collate_fn(batch: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Collate function for VQA DataLoader.
    
    Handles batching of pixel values, input IDs, and labels.
    """
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    input_ids = torch.stack([item["input_ids"] for item in batch])
    attention_mask = torch.stack([item["attention_mask"] for item in batch])
    labels = torch.stack([item["labels"] for item in batch])
    
    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "question_ids": [item["question_id"] for item in batch],
        "questions": [item["question"] for item in batch],
        "answers": [item["answer"] for item in batch],
    }


def create_dataloaders(
    processor: Blip2Processor,
    config,
    seed: int = 42
) -> Tuple[DataLoader, DataLoader]:
    """
    Create train and validation dataloaders.
    
    Args:
        processor: BLIP-2 processor
        config: Configuration object
        seed: Random seed for reproducibility
        
    Returns:
        Tuple of (train_loader, val_loader)
    """
    from src.utils.seed import get_worker_init_fn, get_generator
    
    # Determine sample limits
    max_train = config.data.max_samples_train
    max_val = config.data.max_samples_val
    
    if config.training.smoke_test:
        max_train = config.training.smoke_test_samples
        max_val = config.training.smoke_test_samples // 2
    
    # Create datasets
    train_dataset = VQADataset(
        processor=processor,
        split=config.data.split_train,
        dataset_name=config.data.dataset_name,
        max_samples=max_train,
        max_question_length=config.data.max_question_length,
        max_answer_length=config.data.max_answer_length,
        cache_dir=config.data.cache_dir,
    )
    
    val_dataset = VQADataset(
        processor=processor,
        split=config.data.split_val,
        dataset_name=config.data.dataset_name,
        max_samples=max_val,
        max_question_length=config.data.max_question_length,
        max_answer_length=config.data.max_answer_length,
        cache_dir=config.data.cache_dir,
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.training.batch_size,
        shuffle=True,
        num_workers=config.data.num_workers,
        collate_fn=vqa_collate_fn,
        worker_init_fn=get_worker_init_fn(seed),
        generator=get_generator(seed),
        pin_memory=True,
        drop_last=True,
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.training.batch_size,
        shuffle=False,
        num_workers=config.data.num_workers,
        collate_fn=vqa_collate_fn,
        pin_memory=True,
    )
    
    return train_loader, val_loader
'''

write_file("src/data/vqa_dataset.py", vqa_dataset_py_content)

## Section 6: Answer Vocabulary and Processing

Utilities for answer normalization, vocabulary building, and string matching for VQA evaluation.

In [ ]:
# ============================================================================
# FILE: src/data/answer_vocab.py
# Answer normalization and vocabulary utilities
# ============================================================================

answer_vocab_py_content = '''"""
Answer vocabulary and normalization utilities for VQA.

Provides answer normalization (lowercase, punctuation removal, article stripping)
and optional vocabulary building for classification-based VQA.
"""

import re
import string
from typing import List, Dict, Optional, Set
from collections import Counter


# Common articles to remove for normalization
ARTICLES = {'a', 'an', 'the'}

# Punctuation translation table
PUNCT_TABLE = str.maketrans('', '', string.punctuation)


def normalize_answer(answer: str) -> str:
    """
    Normalize answer string for comparison.
    
    Normalization steps:
    1. Convert to lowercase
    2. Remove punctuation
    3. Remove articles (a, an, the)
    4. Strip whitespace and collapse multiple spaces
    
    Args:
        answer: Raw answer string
        
    Returns:
        Normalized answer string
    """
    # Convert to lowercase
    answer = answer.lower()
    
    # Remove punctuation
    answer = answer.translate(PUNCT_TABLE)
    
    # Remove articles
    words = answer.split()
    words = [w for w in words if w not in ARTICLES]
    
    # Rejoin and strip
    answer = ' '.join(words).strip()
    
    # Collapse multiple spaces
    answer = re.sub(r'\\s+', ' ', answer)
    
    return answer


def exact_match(pred: str, target: str) -> bool:
    """
    Check if prediction exactly matches target.
    
    Args:
        pred: Predicted answer
        target: Ground truth answer
        
    Returns:
        True if exact match
    """
    return pred.strip().lower() == target.strip().lower()


def normalized_match(pred: str, target: str) -> bool:
    """
    Check if normalized prediction matches normalized target.
    
    Args:
        pred: Predicted answer
        target: Ground truth answer
        
    Returns:
        True if normalized match
    """
    return normalize_answer(pred) == normalize_answer(target)


def soft_match(pred: str, target: str) -> float:
    """
    Compute soft match score between prediction and target.
    
    Uses word overlap to compute partial credit.
    
    Args:
        pred: Predicted answer
        target: Ground truth answer
        
    Returns:
        Match score between 0 and 1
    """
    pred_words = set(normalize_answer(pred).split())
    target_words = set(normalize_answer(target).split())
    
    if not target_words:
        return 1.0 if not pred_words else 0.0
    
    overlap = len(pred_words & target_words)
    return overlap / len(target_words)


class AnswerVocabulary:
    """
    Answer vocabulary for classification-based VQA.
    
    Builds a vocabulary from training answers and provides
    encoding/decoding functionality.
    """
    
    def __init__(
        self,
        min_freq: int = 5,
        max_vocab_size: Optional[int] = 3000,
        unk_token: str = "<UNK>"
    ):
        """
        Initialize answer vocabulary.
        
        Args:
            min_freq: Minimum frequency for vocabulary inclusion
            max_vocab_size: Maximum vocabulary size
            unk_token: Token for unknown answers
        """
        self.min_freq = min_freq
        self.max_vocab_size = max_vocab_size
        self.unk_token = unk_token
        
        self.answer_to_idx: Dict[str, int] = {}
        self.idx_to_answer: Dict[int, str] = {}
        self.answer_freq: Counter = Counter()
        self._is_built = False
    
    def build_from_answers(self, answers: List[str]) -> 'AnswerVocabulary':
        """
        Build vocabulary from list of answers.
        
        Args:
            answers: List of answer strings
            
        Returns:
            Self for chaining
        """
        # Count normalized answers
        normalized_answers = [normalize_answer(a) for a in answers]
        self.answer_freq = Counter(normalized_answers)
        
        # Filter by frequency
        filtered = [(a, c) for a, c in self.answer_freq.most_common() 
                    if c >= self.min_freq]
        
        # Limit vocabulary size
        if self.max_vocab_size:
            filtered = filtered[:self.max_vocab_size - 1]  # Reserve space for UNK
        
        # Build mappings
        self.answer_to_idx = {self.unk_token: 0}
        self.idx_to_answer = {0: self.unk_token}
        
        for idx, (answer, _) in enumerate(filtered, start=1):
            self.answer_to_idx[answer] = idx
            self.idx_to_answer[idx] = answer
        
        self._is_built = True
        print(f"📖 Answer vocabulary built: {len(self)} answers")
        
        return self
    
    def encode(self, answer: str) -> int:
        """Encode answer to vocabulary index."""
        normalized = normalize_answer(answer)
        return self.answer_to_idx.get(normalized, 0)  # 0 = UNK
    
    def decode(self, idx: int) -> str:
        """Decode vocabulary index to answer."""
        return self.idx_to_answer.get(idx, self.unk_token)
    
    def __len__(self) -> int:
        return len(self.answer_to_idx)
    
    def __contains__(self, answer: str) -> bool:
        return normalize_answer(answer) in self.answer_to_idx
    
    def save(self, path: str) -> None:
        """Save vocabulary to file."""
        import json
        with open(path, 'w') as f:
            json.dump({
                'answer_to_idx': self.answer_to_idx,
                'min_freq': self.min_freq,
                'max_vocab_size': self.max_vocab_size,
                'unk_token': self.unk_token,
            }, f, indent=2)
    
    @classmethod
    def load(cls, path: str) -> 'AnswerVocabulary':
        """Load vocabulary from file."""
        import json
        with open(path, 'r') as f:
            data = json.load(f)
        
        vocab = cls(
            min_freq=data.get('min_freq', 5),
            max_vocab_size=data.get('max_vocab_size'),
            unk_token=data.get('unk_token', '<UNK>')
        )
        vocab.answer_to_idx = data['answer_to_idx']
        vocab.idx_to_answer = {int(v): k for k, v in vocab.answer_to_idx.items()}
        vocab._is_built = True
        
        return vocab


def get_vqa_accuracy(
    predictions: List[str],
    targets: List[str],
    use_normalized: bool = True
) -> Dict[str, float]:
    """
    Compute VQA accuracy metrics.
    
    Args:
        predictions: List of predicted answers
        targets: List of ground truth answers
        use_normalized: Use normalized matching
        
    Returns:
        Dictionary with accuracy metrics
    """
    assert len(predictions) == len(targets), "Predictions and targets must have same length"
    
    n = len(predictions)
    if n == 0:
        return {"exact_match": 0.0, "normalized_match": 0.0}
    
    exact_matches = sum(exact_match(p, t) for p, t in zip(predictions, targets))
    normalized_matches = sum(normalized_match(p, t) for p, t in zip(predictions, targets))
    
    return {
        "exact_match": exact_matches / n,
        "normalized_match": normalized_matches / n,
        "total_samples": n,
    }
'''

write_file("src/data/answer_vocab.py", answer_vocab_py_content)

## Section 7: BLIP-2 Wrapper Model

Clean wrapper around the BLIP-2 model with:
- Configurable freezing of components
- Forward method returning loss for training
- Generate method for inference with proper prompt handling
- Hooks for Scene Reasoning Module integration

In [ ]:
# ============================================================================
# FILE: src/models/blip2_wrapper.py
# BLIP-2 model wrapper for VQA
# ============================================================================

blip2_wrapper_py_content = '''"""
BLIP-2 Wrapper for Visual Question Answering.

Provides a clean interface around HuggingFace BLIP-2 model with:
- Configurable component freezing
- Training forward pass with loss computation
- Generation for inference with prompt handling
- Integration hooks for Scene Reasoning Module
"""

import torch
import torch.nn as nn
from typing import Dict, Any, Optional, Tuple, List
from transformers import (
    Blip2ForConditionalGeneration,
    Blip2Processor,
    Blip2Config
)


class BLIP2VQAWrapper(nn.Module):
    """
    Wrapper for BLIP-2 model tailored for VQA tasks.
    
    Supports both generative VQA (default) and optional classification mode.
    """
    
    def __init__(
        self,
        model_name: str = "Salesforce/blip2-opt-2.7b",
        freeze_vision_encoder: bool = True,
        freeze_llm: bool = True,
        freeze_qformer: bool = False,
        device_map: str = "auto",
        torch_dtype: torch.dtype = torch.float16,
        scene_reasoning_module: Optional[nn.Module] = None,
        max_new_tokens: int = 16,
        num_beams: int = 3,
    ):
        """
        Initialize BLIP-2 VQA wrapper.
        
        Args:
            model_name: HuggingFace model identifier
            freeze_vision_encoder: Freeze vision encoder weights
            freeze_llm: Freeze language model weights
            freeze_qformer: Freeze Q-Former weights
            device_map: Device mapping for model loading
            torch_dtype: Model precision
            scene_reasoning_module: Optional scene reasoning module
            max_new_tokens: Maximum tokens for generation
            num_beams: Beam search width
        """
        super().__init__()
        
        self.model_name = model_name
        self.max_new_tokens = max_new_tokens
        self.num_beams = num_beams
        
        print(f"🔄 Loading BLIP-2 model: {model_name}")
        
        # Load model and processor
        self.model = Blip2ForConditionalGeneration.from_pretrained(
            model_name,
            device_map=device_map,
            torch_dtype=torch_dtype,
        )
        
        self.processor = Blip2Processor.from_pretrained(model_name)
        
        # Store config
        self.config = self.model.config
        
        # Apply freezing
        self._freeze_components(freeze_vision_encoder, freeze_llm, freeze_qformer)
        
        # Scene reasoning module (integrated between vision and Q-Former)
        self.scene_reasoning = scene_reasoning_module
        
        # Track trainable parameters
        self._log_trainable_params()
    
    def _freeze_components(
        self,
        freeze_vision: bool,
        freeze_llm: bool,
        freeze_qformer: bool
    ) -> None:
        """Freeze model components based on configuration."""
        
        # Freeze vision encoder
        if freeze_vision:
            for param in self.model.vision_model.parameters():
                param.requires_grad = False
            print("   ❄️ Vision encoder frozen")
        
        # Freeze language model
        if freeze_llm:
            for param in self.model.language_model.parameters():
                param.requires_grad = False
            print("   ❄️ Language model frozen")
        
        # Freeze Q-Former
        if freeze_qformer:
            for param in self.model.qformer.parameters():
                param.requires_grad = False
            print("   ❄️ Q-Former frozen")
    
    def _log_trainable_params(self) -> None:
        """Log trainable parameter count."""
        total_params = sum(p.numel() for p in self.parameters())
        trainable_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        
        print(f"   📊 Total params: {total_params / 1e6:.1f}M")
        print(f"   📊 Trainable params: {trainable_params / 1e6:.1f}M ({100*trainable_params/total_params:.1f}%)")
    
    def get_vision_features(
        self,
        pixel_values: torch.Tensor
    ) -> torch.Tensor:
        """
        Extract vision features from image.
        
        Args:
            pixel_values: Image tensor [B, C, H, W]
            
        Returns:
            Vision features [B, num_patches, hidden_dim]
        """
        vision_outputs = self.model.vision_model(
            pixel_values=pixel_values,
            return_dict=True
        )
        
        # Get patch embeddings (excluding CLS token)
        image_embeds = vision_outputs.last_hidden_state
        
        return image_embeds
    
    def apply_scene_reasoning(
        self,
        vision_features: torch.Tensor,
        return_attention: bool = False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Apply scene reasoning module to vision features.
        
        Args:
            vision_features: Vision features [B, num_patches, hidden_dim]
            return_attention: Whether to return attention weights
            
        Returns:
            Enhanced features and optional attention weights
        """
        if self.scene_reasoning is None:
            return vision_features, None
        
        if return_attention:
            enhanced_features, attention_weights = self.scene_reasoning(
                vision_features, return_attention=True
            )
            return enhanced_features, attention_weights
        else:
            enhanced_features = self.scene_reasoning(vision_features)
            return enhanced_features, None
    
    def forward(
        self,
        pixel_values: torch.Tensor,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        return_dict: bool = True,
        **kwargs
    ) -> Dict[str, torch.Tensor]:
        """
        Forward pass for training.
        
        Args:
            pixel_values: Image tensor [B, C, H, W]
            input_ids: Input token IDs [B, seq_len]
            attention_mask: Attention mask [B, seq_len]
            labels: Target labels for loss computation [B, seq_len]
            return_dict: Return as dictionary
            
        Returns:
            Dictionary containing loss and logits
        """
        # Standard BLIP-2 forward pass
        outputs = self.model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True,
        )
        
        if return_dict:
            return {
                "loss": outputs.loss,
                "logits": outputs.logits if hasattr(outputs, 'logits') else None,
            }
        
        return outputs
    
    def forward_with_scene_reasoning(
        self,
        pixel_values: torch.Tensor,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        return_attention: bool = False,
    ) -> Dict[str, torch.Tensor]:
        """
        Forward pass with scene reasoning module integration.
        
        Note: This requires modifying the internal BLIP-2 forward pass.
        For simplicity, we extract features, apply reasoning, then continue.
        """
        # Extract vision features
        vision_features = self.get_vision_features(pixel_values)
        
        # Apply scene reasoning
        enhanced_features, scene_attention = self.apply_scene_reasoning(
            vision_features, return_attention=return_attention
        )
        
        # Continue with Q-Former and LLM
        # Note: Full integration would require modifying BLIP-2 internals
        # Here we use the standard forward but the scene_reasoning enhances
        # features that can be used for analysis
        
        outputs = self.model(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            return_dict=True,
        )
        
        result = {
            "loss": outputs.loss,
            "logits": outputs.logits if hasattr(outputs, 'logits') else None,
            "enhanced_vision_features": enhanced_features,
        }
        
        if return_attention and scene_attention is not None:
            result["scene_attention"] = scene_attention
        
        return result
    
    @torch.no_grad()
    def generate(
        self,
        pixel_values: torch.Tensor,
        input_ids: torch.Tensor,
        attention_mask: Optional[torch.Tensor] = None,
        max_new_tokens: Optional[int] = None,
        num_beams: Optional[int] = None,
        **kwargs
    ) -> List[str]:
        """
        Generate answers for VQA.
        
        Args:
            pixel_values: Image tensor [B, C, H, W]
            input_ids: Input token IDs (question prompt) [B, seq_len]
            attention_mask: Attention mask [B, seq_len]
            max_new_tokens: Maximum new tokens to generate
            num_beams: Number of beams for beam search
            
        Returns:
            List of generated answer strings
        """
        max_new_tokens = max_new_tokens or self.max_new_tokens
        num_beams = num_beams or self.num_beams
        
        # Generate
        generated_ids = self.model.generate(
            pixel_values=pixel_values,
            input_ids=input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            num_beams=num_beams,
            do_sample=False,
            **kwargs
        )
        
        # Decode
        generated_texts = self.processor.batch_decode(
            generated_ids,
            skip_special_tokens=True
        )
        
        # Post-process: remove prompt if present
        answers = []
        for text in generated_texts:
            # Strip common prompt artifacts
            text = text.strip()
            if "Answer:" in text:
                text = text.split("Answer:")[-1].strip()
            answers.append(text)
        
        return answers
    
    def get_processor(self) -> Blip2Processor:
        """Return the BLIP-2 processor."""
        return self.processor


def create_blip2_model(config) -> BLIP2VQAWrapper:
    """
    Factory function to create BLIP-2 model from config.
    
    Args:
        config: Model configuration object
        
    Returns:
        Configured BLIP2VQAWrapper
    """
    # Import scene reasoning if needed
    scene_module = None
    if config.model.use_scene_reasoning:
        from src.models.scene_reasoning import SceneReasoningModule
        scene_module = SceneReasoningModule(
            hidden_dim=config.model.scene_reasoning_dim,
            num_heads=config.model.scene_reasoning_heads,
            num_layers=config.model.scene_reasoning_layers,
            use_spatial_encoding=config.model.use_spatial_encoding,
            use_relation_attention=config.model.use_relation_attention,
            spatial_dim=config.model.spatial_encoding_dim,
        )
    
    model = BLIP2VQAWrapper(
        model_name=config.model.model_name,
        freeze_vision_encoder=config.model.freeze_vision_encoder,
        freeze_llm=config.model.freeze_llm,
        freeze_qformer=config.model.freeze_qformer,
        scene_reasoning_module=scene_module,
        max_new_tokens=config.model.max_new_tokens,
        num_beams=config.model.num_beams,
    )
    
    return model
'''

write_file("src/models/blip2_wrapper.py", blip2_wrapper_py_content)

## Section 8: VQA Head Implementation

Optional classification head for vocabulary-based VQA mode.

In [ ]:
# ============================================================================
# FILE: src/models/vqa_head.py
# VQA classification head (optional)
# ============================================================================

vqa_head_py_content = '''"""
VQA Classification Head.

Optional classification head for vocabulary-based VQA mode.
Maps fused vision-language features to answer vocabulary logits.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional, Tuple


class VQAClassificationHead(nn.Module):
    """
    Classification head for VQA.
    
    Projects fused features to answer vocabulary logits.
    """
    
    def __init__(
        self,
        input_dim: int,
        hidden_dim: int = 1024,
        vocab_size: int = 3000,
        dropout: float = 0.3,
        use_layer_norm: bool = True,
    ):
        """
        Initialize VQA classification head.
        
        Args:
            input_dim: Input feature dimension
            hidden_dim: Hidden layer dimension
            vocab_size: Answer vocabulary size
            dropout: Dropout probability
            use_layer_norm: Whether to use layer normalization
        """
        super().__init__()
        
        self.use_layer_norm = use_layer_norm
        
        # Feature projection
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.classifier = nn.Linear(hidden_dim, vocab_size)
        
        # Normalization and dropout
        self.dropout = nn.Dropout(dropout)
        if use_layer_norm:
            self.ln1 = nn.LayerNorm(hidden_dim)
            self.ln2 = nn.LayerNorm(hidden_dim)
        
        # Initialize weights
        self._init_weights()
    
    def _init_weights(self) -> None:
        """Initialize layer weights."""
        for module in [self.fc1, self.fc2, self.classifier]:
            nn.init.xavier_uniform_(module.weight)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
    
    def forward(
        self,
        features: torch.Tensor,
        return_features: bool = False
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Forward pass.
        
        Args:
            features: Input features [B, input_dim]
            return_features: Whether to return intermediate features
            
        Returns:
            Tuple of (logits, optional features)
        """
        # First projection
        x = self.fc1(features)
        if self.use_layer_norm:
            x = self.ln1(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        # Second projection
        x = self.fc2(x)
        if self.use_layer_norm:
            x = self.ln2(x)
        x = F.gelu(x)
        x = self.dropout(x)
        
        # Classification
        logits = self.classifier(x)
        
        if return_features:
            return logits, x
        return logits, None
    
    def predict(self, features: torch.Tensor) -> torch.Tensor:
        """
        Predict answer indices.
        
        Args:
            features: Input features [B, input_dim]
            
        Returns:
            Predicted answer indices [B]
        """
        logits, _ = self.forward(features)
        return logits.argmax(dim=-1)
    
    def predict_topk(
        self,
        features: torch.Tensor,
        k: int = 5
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Predict top-k answer indices and scores.
        
        Args:
            features: Input features [B, input_dim]
            k: Number of top predictions
            
        Returns:
            Tuple of (indices [B, k], scores [B, k])
        """
        logits, _ = self.forward(features)
        probs = F.softmax(logits, dim=-1)
        scores, indices = probs.topk(k, dim=-1)
        return indices, scores


class MultimodalFusion(nn.Module):
    """
    Multimodal fusion module for combining vision and language features.
    """
    
    def __init__(
        self,
        vision_dim: int,
        language_dim: int,
        output_dim: int,
        fusion_type: str = "concat",
        dropout: float = 0.1,
    ):
        """
        Initialize multimodal fusion.
        
        Args:
            vision_dim: Vision feature dimension
            language_dim: Language feature dimension
            output_dim: Output dimension
            fusion_type: Type of fusion ("concat", "multiply", "add")
            dropout: Dropout probability
        """
        super().__init__()
        
        self.fusion_type = fusion_type
        
        if fusion_type == "concat":
            self.projection = nn.Linear(vision_dim + language_dim, output_dim)
        elif fusion_type in ["multiply", "add"]:
            # Project both to same dimension first
            self.vision_proj = nn.Linear(vision_dim, output_dim)
            self.language_proj = nn.Linear(language_dim, output_dim)
            self.projection = nn.Linear(output_dim, output_dim)
        else:
            raise ValueError(f"Unknown fusion type: {fusion_type}")
        
        self.dropout = nn.Dropout(dropout)
        self.layer_norm = nn.LayerNorm(output_dim)
    
    def forward(
        self,
        vision_features: torch.Tensor,
        language_features: torch.Tensor
    ) -> torch.Tensor:
        """
        Fuse vision and language features.
        
        Args:
            vision_features: Vision features [B, vision_dim]
            language_features: Language features [B, language_dim]
            
        Returns:
            Fused features [B, output_dim]
        """
        if self.fusion_type == "concat":
            combined = torch.cat([vision_features, language_features], dim=-1)
            fused = self.projection(combined)
        elif self.fusion_type == "multiply":
            v = self.vision_proj(vision_features)
            l = self.language_proj(language_features)
            fused = self.projection(v * l)
        elif self.fusion_type == "add":
            v = self.vision_proj(vision_features)
            l = self.language_proj(language_features)
            fused = self.projection(v + l)
        
        fused = self.dropout(fused)
        fused = self.layer_norm(fused)
        
        return fused
'''

write_file("src/models/vqa_head.py", vqa_head_py_content)

## Section 9: Scene Reasoning Module

The core contribution - a modular Scene Reasoning Module with:
- Relation-aware self-attention for modeling object relationships
- 2D spatial relative position encodings for spatial reasoning
- Interpretable attention weights for visualization
- On/off toggle for ablation studies

In [ ]:
# ============================================================================
# FILE: src/models/scene_reasoning.py
# Scene Reasoning Module with spatial and relational attention
# ============================================================================

scene_reasoning_py_content = '''"""
Scene Reasoning Module for enhanced spatial and relational understanding.

This module processes vision features from BLIP-2's vision encoder and enhances
them with explicit spatial and relational reasoning capabilities.

Key Components:
1. Spatial Position Encodings: 2D relative position encodings for patches
2. Relation-Aware Self-Attention: Models relationships between image regions
3. Interpretability: Exposes attention weights for visualization
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass, field
from typing import Optional, Tuple, Dict


@dataclass
class SceneReasoningConfig:
    """Configuration for Scene Reasoning Module."""
    hidden_dim: int = 768
    num_heads: int = 12
    num_layers: int = 2
    mlp_ratio: float = 4.0
    dropout: float = 0.1
    use_spatial_encoding: bool = True
    use_relation_attention: bool = True
    spatial_dim: int = 64
    max_positions: int = 24
    
    @classmethod
    def from_dict(cls, d: Dict) -> 'SceneReasoningConfig':
        return cls(**{k: v for k, v in d.items() if k in cls.__dataclass_fields__})


class SpatialPositionEncoding(nn.Module):
    """
    2D Spatial Position Encoding for image patches.
    
    Creates learnable relative position encodings based on
    2D spatial relationships between patches.
    """
    
    def __init__(
        self,
        hidden_dim: int = 768,
        spatial_dim: int = 64,
        max_positions: int = 24,  # Max patches per dimension (for 224x224 with 14x14 patches)
    ):
        """
        Initialize spatial position encoding.
        
        Args:
            hidden_dim: Model hidden dimension
            spatial_dim: Dimension of spatial encodings
            max_positions: Maximum positions per spatial dimension
        """
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.spatial_dim = spatial_dim
        self.max_positions = max_positions
        
        # Learnable position embeddings for row and column
        self.row_embed = nn.Embedding(max_positions, spatial_dim // 2)
        self.col_embed = nn.Embedding(max_positions, spatial_dim // 2)
        
        # Projection to hidden dimension
        self.position_proj = nn.Linear(spatial_dim, hidden_dim)
        
        # Relative position bias for attention
        self.relative_position_bias = nn.Parameter(
            torch.zeros(2 * max_positions - 1, 2 * max_positions - 1)
        )
        nn.init.trunc_normal_(self.relative_position_bias, std=0.02)
    
    def get_absolute_positions(
        self,
        batch_size: int,
        num_patches: int,
        device: torch.device
    ) -> torch.Tensor:
        """
        Compute absolute 2D position encodings.
        
        Args:
            batch_size: Batch size
            num_patches: Total number of patches
            device: Device for tensors
            
        Returns:
            Position encodings [B, num_patches, hidden_dim]
        """
        # Assume square patch grid
        grid_size = int(math.sqrt(num_patches))
        
        # Create position indices
        rows = torch.arange(grid_size, device=device)
        cols = torch.arange(grid_size, device=device)
        
        # Get embeddings
        row_emb = self.row_embed(rows)  # [grid_size, spatial_dim//2]
        col_emb = self.col_embed(cols)  # [grid_size, spatial_dim//2]
        
        # Create 2D position grid
        row_grid, col_grid = torch.meshgrid(rows, cols, indexing='ij')
        row_pos = self.row_embed(row_grid.reshape(-1))  # [num_patches, spatial_dim//2]
        col_pos = self.col_embed(col_grid.reshape(-1))  # [num_patches, spatial_dim//2]
        
        # Concatenate row and column positions
        positions = torch.cat([row_pos, col_pos], dim=-1)  # [num_patches, spatial_dim]
        
        # Project to hidden dimension
        positions = self.position_proj(positions)  # [num_patches, hidden_dim]
        
        # Expand for batch
        positions = positions.unsqueeze(0).expand(batch_size, -1, -1)
        
        return positions
    
    def get_relative_position_bias(
        self,
        num_patches: int,
        device: torch.device
    ) -> torch.Tensor:
        """
        Compute relative position bias for attention.
        
        Args:
            num_patches: Total number of patches
            device: Device for tensors
            
        Returns:
            Relative position bias [num_patches, num_patches]
        """
        grid_size = int(math.sqrt(num_patches))
        
        # Create coordinate grids
        coords = torch.stack(torch.meshgrid(
            torch.arange(grid_size, device=device),
            torch.arange(grid_size, device=device),
            indexing='ij'
        ), dim=-1).reshape(-1, 2)  # [num_patches, 2]
        
        # Compute relative positions
        relative_coords = coords[:, None, :] - coords[None, :, :]  # [N, N, 2]
        
        # Shift to positive indices
        relative_coords[:, :, 0] += self.max_positions - 1
        relative_coords[:, :, 1] += self.max_positions - 1
        
        # Get bias values
        relative_position_index = relative_coords[:, :, 0] * (2 * self.max_positions - 1) + relative_coords[:, :, 1]
        relative_position_index = relative_position_index.clamp(
            0, self.relative_position_bias.numel() - 1
        )
        
        bias = self.relative_position_bias.view(-1)[relative_position_index.view(-1)]
        bias = bias.view(num_patches, num_patches)
        
        return bias
    
    def forward(
        self,
        x: torch.Tensor,
        return_bias: bool = True
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Add spatial position encodings to input.
        
        Args:
            x: Input features [B, num_patches, hidden_dim]
            return_bias: Whether to return relative position bias
            
        Returns:
            Tuple of (position-enhanced features, optional relative bias)
        """
        batch_size, num_patches, _ = x.shape
        
        # Get absolute positions
        positions = self.get_absolute_positions(batch_size, num_patches, x.device)
        
        # Add to input
        x = x + positions
        
        # Get relative bias for attention
        if return_bias:
            bias = self.get_relative_position_bias(num_patches, x.device)
            return x, bias
        
        return x, None


class RelationAwareSelfAttention(nn.Module):
    """
    Relation-Aware Self-Attention with optional learned adjacency.
    
    Enhances standard self-attention with:
    1. Relative position bias
    2. Optional learned adjacency matrix for relation modeling
    """
    
    def __init__(
        self,
        hidden_dim: int = 768,
        num_heads: int = 8,
        dropout: float = 0.1,
        use_relative_bias: bool = True,
        use_learned_adjacency: bool = False,
    ):
        """
        Initialize relation-aware self-attention.
        
        Args:
            hidden_dim: Hidden dimension
            num_heads: Number of attention heads
            dropout: Dropout probability
            use_relative_bias: Use relative position bias
            use_learned_adjacency: Use learned adjacency matrix
        """
        super().__init__()
        
        self.hidden_dim = hidden_dim
        self.num_heads = num_heads
        self.head_dim = hidden_dim // num_heads
        self.scale = self.head_dim ** -0.5
        
        self.use_relative_bias = use_relative_bias
        self.use_learned_adjacency = use_learned_adjacency
        
        # QKV projection
        self.qkv = nn.Linear(hidden_dim, hidden_dim * 3)
        self.proj = nn.Linear(hidden_dim, hidden_dim)
        
        self.dropout = nn.Dropout(dropout)
        
        # Learned adjacency (initialized later based on input size)
        self.adjacency = None
        self._adjacency_initialized = False
    
    def _init_adjacency(self, num_patches: int, device: torch.device):
        """Initialize learned adjacency matrix."""
        if not self._adjacency_initialized and self.use_learned_adjacency:
            self.adjacency = nn.Parameter(
                torch.zeros(num_patches, num_patches, device=device)
            )
            nn.init.xavier_uniform_(self.adjacency)
            self._adjacency_initialized = True
    
    def forward(
        self,
        x: torch.Tensor,
        relative_position_bias: Optional[torch.Tensor] = None,
        return_attention: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """
        Forward pass.
        
        Args:
            x: Input features [B, N, hidden_dim]
            relative_position_bias: Optional bias [N, N]
            return_attention: Whether to return attention weights
            
        Returns:
            Tuple of (output features, optional attention weights)
        """
        B, N, C = x.shape
        
        # Initialize adjacency if needed
        if self.use_learned_adjacency:
            self._init_adjacency(N, x.device)
        
        # Compute QKV
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim)
        qkv = qkv.permute(2, 0, 3, 1, 4)  # [3, B, heads, N, head_dim]
        q, k, v = qkv[0], qkv[1], qkv[2]
        
        # Compute attention scores
        attn = (q @ k.transpose(-2, -1)) * self.scale  # [B, heads, N, N]
        
        # Add relative position bias
        if relative_position_bias is not None and self.use_relative_bias:
            attn = attn + relative_position_bias.unsqueeze(0).unsqueeze(0)
        
        # Add learned adjacency
        if self.adjacency is not None:
            attn = attn + self.adjacency.unsqueeze(0).unsqueeze(0)
        
        # Softmax and dropout
        attn_weights = F.softmax(attn, dim=-1)
        attn_weights = self.dropout(attn_weights)
        
        # Apply attention to values
        x = (attn_weights @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        
        if return_attention:
            # Average attention over heads for visualization
            avg_attention = attn_weights.mean(dim=1)  # [B, N, N]
            return x, avg_attention
        
        return x, None


class SceneReasoningLayer(nn.Module):
    """
    Single layer of Scene Reasoning with attention and FFN.
    """
    
    def __init__(
        self,
        hidden_dim: int = 768,
        num_heads: int = 8,
        mlp_ratio: float = 4.0,
        dropout: float = 0.1,
        use_relative_bias: bool = True,
        use_learned_adjacency: bool = False,
    ):
        """Initialize scene reasoning layer."""
        super().__init__()
        
        self.norm1 = nn.LayerNorm(hidden_dim)
        self.attention = RelationAwareSelfAttention(
            hidden_dim=hidden_dim,
            num_heads=num_heads,
            dropout=dropout,
            use_relative_bias=use_relative_bias,
            use_learned_adjacency=use_learned_adjacency,
        )
        
        self.norm2 = nn.LayerNorm(hidden_dim)
        mlp_hidden = int(hidden_dim * mlp_ratio)
        self.mlp = nn.Sequential(
            nn.Linear(hidden_dim, mlp_hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(mlp_hidden, hidden_dim),
            nn.Dropout(dropout),
        )
    
    def forward(
        self,
        x: torch.Tensor,
        relative_position_bias: Optional[torch.Tensor] = None,
        return_attention: bool = False,
    ) -> Tuple[torch.Tensor, Optional[torch.Tensor]]:
        """Forward pass with residual connections."""
        
        # Self-attention with residual
        residual = x
        x = self.norm1(x)
        attn_out, attn_weights = self.attention(
            x, relative_position_bias, return_attention
        )
        x = residual + attn_out
        
        # MLP with residual
        x = x + self.mlp(self.norm2(x))
        
        return x, attn_weights


class SceneReasoningModule(nn.Module):
    """
    Complete Scene Reasoning Module.
    
    Enhances vision features with spatial and relational reasoning
    before they are processed by the Q-Former.
    """
    
    def __init__(
        self,
        config: Optional[SceneReasoningConfig] = None,
        hidden_dim: int = 768,
        num_heads: int = 8,
        num_layers: int = 2,
        mlp_ratio: float = 4.0,
        dropout: float = 0.1,
        use_spatial_encoding: bool = True,
        use_relation_attention: bool = True,
        spatial_dim: int = 64,
    ):
        """
        Initialize Scene Reasoning Module.
        
        Args:
            config: Optional SceneReasoningConfig (if provided, overrides other params)
            hidden_dim: Hidden dimension (should match vision encoder output)
            num_heads: Number of attention heads
            num_layers: Number of reasoning layers
            mlp_ratio: MLP hidden dimension ratio
            dropout: Dropout probability
            use_spatial_encoding: Enable 2D spatial position encodings
            use_relation_attention: Enable relation-aware attention
            spatial_dim: Spatial encoding dimension
        """
        super().__init__()
        
        # If config provided, use its values
        if config is not None:
            hidden_dim = config.hidden_dim
            num_heads = config.num_heads
            num_layers = config.num_layers
            mlp_ratio = config.mlp_ratio
            dropout = config.dropout
            use_spatial_encoding = config.use_spatial_encoding
            use_relation_attention = config.use_relation_attention
            spatial_dim = config.spatial_dim
        
        self.hidden_dim = hidden_dim
        self.use_spatial_encoding = use_spatial_encoding
        self.use_relation_attention = use_relation_attention
        
        # Input projection (in case hidden_dim doesn't match)
        self.input_proj = nn.Linear(hidden_dim, hidden_dim)
        
        # Spatial position encoding
        self.spatial_encoding = None
        if use_spatial_encoding:
            self.spatial_encoding = SpatialPositionEncoding(
                hidden_dim=hidden_dim,
                spatial_dim=spatial_dim,
            )
        
        # Reasoning layers
        self.layers = nn.ModuleList([
            SceneReasoningLayer(
                hidden_dim=hidden_dim,
                num_heads=num_heads,
                mlp_ratio=mlp_ratio,
                dropout=dropout,
                use_relative_bias=use_spatial_encoding,
                use_learned_adjacency=use_relation_attention,
            )
            for _ in range(num_layers)
        ])
        
        # Output normalization
        self.norm = nn.LayerNorm(hidden_dim)
        
        print(f"🧠 Scene Reasoning Module initialized:")
        print(f"   Layers: {num_layers}, Heads: {num_heads}")
        print(f"   Spatial encoding: {use_spatial_encoding}")
        print(f"   Relation attention: {use_relation_attention}")
    
    def forward(
        self,
        x: torch.Tensor,
        return_attention: bool = False,
    ) -> Tuple[torch.Tensor, Optional[Dict[str, torch.Tensor]]]:
        """
        Process vision features with scene reasoning.
        
        Args:
            x: Vision features [B, num_patches, hidden_dim]
            return_attention: Whether to return attention weights
            
        Returns:
            Tuple of (enhanced features, optional attention dict)
        """
        # Input projection
        x = self.input_proj(x)
        
        # Add spatial encodings and get relative bias
        relative_bias = None
        if self.spatial_encoding is not None:
            x, relative_bias = self.spatial_encoding(x, return_bias=True)
        
        # Process through reasoning layers
        attention_weights = {} if return_attention else None
        
        for i, layer in enumerate(self.layers):
            x, attn = layer(x, relative_bias, return_attention)
            if return_attention and attn is not None:
                attention_weights[f"layer_{i}"] = attn
        
        # Output normalization
        x = self.norm(x)
        
        if return_attention:
            return x, attention_weights
        
        return x, None
    
    def get_attention_rollout(
        self,
        attention_weights: Dict[str, torch.Tensor]
    ) -> torch.Tensor:
        """
        Compute attention rollout for interpretability.
        
        Args:
            attention_weights: Dictionary of layer attention weights
            
        Returns:
            Aggregated attention map
        """
        # Start with identity
        result = None
        
        for key in sorted(attention_weights.keys()):
            attn = attention_weights[key]  # [B, N, N]
            
            # Add identity and renormalize (attention rollout)
            attn = 0.5 * attn + 0.5 * torch.eye(
                attn.size(-1), device=attn.device
            ).unsqueeze(0)
            
            if result is None:
                result = attn
            else:
                result = attn @ result
        
        return result
'''

write_file("src/models/scene_reasoning.py", scene_reasoning_py_content)

## Section 10: Loss Functions

Loss functions for generative VQA training with label smoothing and proper masking.

In [ ]:
# ============================================================================
# FILE: src/training/losses.py
# Loss functions for VQA training
# ============================================================================

losses_py_content = '''"""
Loss functions for VQA training.

Includes cross-entropy loss with label smoothing and proper padding handling.
"""

import torch
import torch.nn as nn
import torch.nn.functional as F
from typing import Optional


class LabelSmoothingCrossEntropy(nn.Module):
    """
    Cross-entropy loss with label smoothing.
    
    Helpful for preventing overconfident predictions and improving generalization.
    """
    
    def __init__(
        self,
        smoothing: float = 0.1,
        reduction: str = 'mean',
        ignore_index: int = -100,
    ):
        """
        Initialize label smoothing cross-entropy.
        
        Args:
            smoothing: Label smoothing factor (0 = no smoothing)
            reduction: Loss reduction method ('mean', 'sum', 'none')
            ignore_index: Index to ignore in loss computation
        """
        super().__init__()
        
        self.smoothing = smoothing
        self.reduction = reduction
        self.ignore_index = ignore_index
    
    def forward(
        self,
        logits: torch.Tensor,
        targets: torch.Tensor,
    ) -> torch.Tensor:
        """
        Compute label-smoothed cross-entropy loss.
        
        Args:
            logits: Model predictions [B, seq_len, vocab_size]
            targets: Target indices [B, seq_len]
            
        Returns:
            Computed loss
        """
        vocab_size = logits.size(-1)
        
        # Reshape for loss computation
        logits = logits.view(-1, vocab_size)
        targets = targets.view(-1)
        
        # Create smoothed targets
        with torch.no_grad():
            # Start with uniform distribution
            smooth_targets = torch.full_like(
                logits, self.smoothing / (vocab_size - 1)
            )
            
            # Set the true class probability
            smooth_targets.scatter_(
                1, targets.unsqueeze(1),
                1.0 - self.smoothing
            )
            
            # Mask ignored indices
            mask = targets != self.ignore_index
            smooth_targets = smooth_targets * mask.unsqueeze(1)
        
        # Compute cross-entropy with smoothed targets
        log_probs = F.log_softmax(logits, dim=-1)
        loss = -torch.sum(smooth_targets * log_probs, dim=-1)
        
        # Apply mask
        loss = loss * mask
        
        # Reduction
        if self.reduction == 'mean':
            return loss.sum() / mask.sum().clamp(min=1)
        elif self.reduction == 'sum':
            return loss.sum()
        else:
            return loss


class VQALoss(nn.Module):
    """
    Combined loss for VQA training.
    
    Supports both generative (language model) and classification modes.
    """
    
    def __init__(
        self,
        mode: str = 'generative',
        label_smoothing: float = 0.0,
        ignore_index: int = -100,
    ):
        """
        Initialize VQA loss.
        
        Args:
            mode: Loss mode ('generative' or 'classification')
            label_smoothing: Label smoothing factor
            ignore_index: Index to ignore
        """
        super().__init__()
        
        self.mode = mode
        
        if label_smoothing > 0:
            self.criterion = LabelSmoothingCrossEntropy(
                smoothing=label_smoothing,
                ignore_index=ignore_index,
            )
        else:
            self.criterion = nn.CrossEntropyLoss(
                ignore_index=ignore_index,
                reduction='mean',
            )
    
    def forward(
        self,
        logits: torch.Tensor,
        targets: torch.Tensor,
        mask: Optional[torch.Tensor] = None,
    ) -> torch.Tensor:
        """
        Compute VQA loss.
        
        Args:
            logits: Model predictions
            targets: Target labels
            mask: Optional attention mask
            
        Returns:
            Computed loss
        """
        if self.mode == 'generative':
            # For language model: [B, seq_len, vocab_size]
            if logits.dim() == 3:
                B, S, V = logits.shape
                logits = logits.view(B * S, V)
                targets = targets.view(B * S)
        
        return self.criterion(logits, targets)


def compute_accuracy(
    logits: torch.Tensor,
    targets: torch.Tensor,
    ignore_index: int = -100,
) -> float:
    """
    Compute token-level accuracy.
    
    Args:
        logits: Model predictions [B, seq_len, vocab_size] or [B, vocab_size]
        targets: Target indices [B, seq_len] or [B]
        ignore_index: Index to ignore
        
    Returns:
        Accuracy as float
    """
    if logits.dim() == 3:
        predictions = logits.argmax(dim=-1)  # [B, seq_len]
    else:
        predictions = logits.argmax(dim=-1)  # [B]
    
    # Mask ignored positions
    mask = targets != ignore_index
    
    correct = (predictions == targets) & mask
    accuracy = correct.sum().float() / mask.sum().clamp(min=1)
    
    return accuracy.item()
'''

with open("/content/VLM_Thesis/src/training/losses.py", 'w') as f:
    f.write(losses_py_content)
print("✅ Created: src/training/losses.py")

## Section 11: Metrics Implementation

VQA evaluation metrics: exact match, normalized match, and running averages for training logs.

In [ ]:
# ============================================================================
# FILE: src/training/metrics.py
# VQA evaluation metrics
# ============================================================================

metrics_py_content = '''"""
Evaluation metrics for VQA.

Implements exact match, normalized match, and metric tracking utilities.
"""

from typing import Dict, List, Optional, Tuple
from collections import defaultdict
import numpy as np


class AverageMeter:
    """Tracks running average of a metric."""
    
    def __init__(self, name: str = "metric"):
        self.name = name
        self.reset()
    
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
    
    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count if self.count > 0 else 0
    
    def __repr__(self):
        return f"{self.name}: {self.avg:.4f}"


class MetricTracker:
    """Tracks multiple metrics during training."""
    
    def __init__(self, metrics: List[str]):
        self.metrics = {name: AverageMeter(name) for name in metrics}
    
    def update(self, values: Dict[str, float], n: int = 1):
        for name, val in values.items():
            if name in self.metrics:
                self.metrics[name].update(val, n)
    
    def reset(self):
        for meter in self.metrics.values():
            meter.reset()
    
    def get_averages(self) -> Dict[str, float]:
        return {name: meter.avg for name, meter in self.metrics.items()}
    
    def __repr__(self):
        return " | ".join(str(m) for m in self.metrics.values())


def normalize_answer(answer: str) -> str:
    """
    Normalize answer for comparison.
    
    Steps:
    1. Lowercase
    2. Remove punctuation
    3. Remove articles (a, an, the)
    4. Strip whitespace
    """
    import string
    import re
    
    answer = answer.lower()
    
    # Remove punctuation
    answer = answer.translate(str.maketrans('', '', string.punctuation))
    
    # Remove articles
    articles = {'a', 'an', 'the'}
    words = answer.split()
    words = [w for w in words if w not in articles]
    
    # Rejoin and clean
    answer = ' '.join(words).strip()
    answer = re.sub(r'\\s+', ' ', answer)
    
    return answer


def exact_match_score(prediction: str, target: str) -> float:
    """Compute exact match (case-insensitive)."""
    return float(prediction.strip().lower() == target.strip().lower())


def normalized_match_score(prediction: str, target: str) -> float:
    """Compute normalized match score."""
    return float(normalize_answer(prediction) == normalize_answer(target))


def compute_vqa_metrics(
    predictions: List[str],
    targets: List[str],
) -> Dict[str, float]:
    """
    Compute VQA metrics for a batch.
    
    Args:
        predictions: List of predicted answers
        targets: List of ground truth answers
        
    Returns:
        Dictionary of metric values
    """
    assert len(predictions) == len(targets)
    n = len(predictions)
    
    if n == 0:
        return {
            "exact_match": 0.0,
            "normalized_match": 0.0,
        }
    
    exact_matches = sum(
        exact_match_score(p, t) for p, t in zip(predictions, targets)
    )
    normalized_matches = sum(
        normalized_match_score(p, t) for p, t in zip(predictions, targets)
    )
    
    return {
        "exact_match": exact_matches / n,
        "normalized_match": normalized_matches / n,
    }


def compute_per_type_metrics(
    predictions: List[str],
    targets: List[str],
    question_types: Optional[List[str]] = None,
) -> Dict[str, Dict[str, float]]:
    """
    Compute metrics per question type.
    
    Args:
        predictions: Predicted answers
        targets: Ground truth answers
        question_types: Optional question types (e.g., "what", "how", "where")
        
    Returns:
        Dictionary mapping question type to metrics
    """
    if question_types is None:
        # Infer simple types from first word
        question_types = []
        for t in targets:  # This is a simplification
            question_types.append("general")
    
    # Group by type
    type_results = defaultdict(lambda: {"correct": 0, "total": 0})
    
    for pred, target, qtype in zip(predictions, targets, question_types):
        is_correct = normalized_match_score(pred, target)
        type_results[qtype]["correct"] += is_correct
        type_results[qtype]["total"] += 1
    
    # Compute accuracy per type
    metrics_per_type = {}
    for qtype, counts in type_results.items():
        if counts["total"] > 0:
            metrics_per_type[qtype] = {
                "accuracy": counts["correct"] / counts["total"],
                "total": counts["total"],
            }
    
    return metrics_per_type


class VQAEvaluator:
    """
    Comprehensive VQA evaluator.
    
    Collects predictions and computes final metrics.
    """
    
    def __init__(self):
        self.predictions = []
        self.targets = []
        self.question_ids = []
        self.questions = []
    
    def add_batch(
        self,
        predictions: List[str],
        targets: List[str],
        question_ids: Optional[List] = None,
        questions: Optional[List[str]] = None,
    ):
        """Add a batch of predictions."""
        self.predictions.extend(predictions)
        self.targets.extend(targets)
        
        if question_ids is not None:
            self.question_ids.extend(question_ids)
        if questions is not None:
            self.questions.extend(questions)
    
    def compute_metrics(self) -> Dict[str, float]:
        """Compute final metrics."""
        return compute_vqa_metrics(self.predictions, self.targets)
    
    def get_results_df(self):
        """Get results as a list of dictionaries."""
        results = []
        for i, (pred, target) in enumerate(zip(self.predictions, self.targets)):
            result = {
                "index": i,
                "prediction": pred,
                "target": target,
                "exact_match": exact_match_score(pred, target),
                "normalized_match": normalized_match_score(pred, target),
            }
            if i < len(self.question_ids):
                result["question_id"] = self.question_ids[i]
            if i < len(self.questions):
                result["question"] = self.questions[i]
            results.append(result)
        return results
    
    def reset(self):
        """Reset evaluator."""
        self.predictions = []
        self.targets = []
        self.question_ids = []
        self.questions = []
'''

with open("/content/VLM_Thesis/src/training/metrics.py", 'w') as f:
    f.write(metrics_py_content)
print("✅ Created: src/training/metrics.py")

## Section 12: Learning Rate Schedulers

Learning rate schedulers with warmup support for stable training.

In [ ]:
# ============================================================================
# FILE: src/training/schedulers.py
# Learning rate schedulers with warmup
# ============================================================================

schedulers_py_content = '''"""
Learning rate schedulers with warmup support.

Includes linear warmup, cosine annealing, and step decay options.
"""

import math
from typing import Optional
import torch
from torch.optim import Optimizer
from torch.optim.lr_scheduler import LambdaLR


def get_linear_warmup_scheduler(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    last_epoch: int = -1,
) -> LambdaLR:
    """
    Linear warmup followed by linear decay.
    
    Args:
        optimizer: Optimizer to schedule
        num_warmup_steps: Number of warmup steps
        num_training_steps: Total training steps
        last_epoch: Last epoch for resuming
        
    Returns:
        LambdaLR scheduler
    """
    def lr_lambda(current_step: int) -> float:
        if current_step < num_warmup_steps:
            # Linear warmup
            return float(current_step) / float(max(1, num_warmup_steps))
        # Linear decay
        return max(
            0.0,
            float(num_training_steps - current_step) /
            float(max(1, num_training_steps - num_warmup_steps))
        )
    
    return LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)


def get_cosine_warmup_scheduler(
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    num_cycles: float = 0.5,
    last_epoch: int = -1,
) -> LambdaLR:
    """
    Linear warmup followed by cosine annealing.
    
    Args:
        optimizer: Optimizer to schedule
        num_warmup_steps: Number of warmup steps
        num_training_steps: Total training steps
        num_cycles: Number of cosine cycles (0.5 = half cycle)
        last_epoch: Last epoch for resuming
        
    Returns:
        LambdaLR scheduler
    """
    def lr_lambda(current_step: int) -> float:
        if current_step < num_warmup_steps:
            # Linear warmup
            return float(current_step) / float(max(1, num_warmup_steps))
        
        # Cosine annealing
        progress = float(current_step - num_warmup_steps) / float(
            max(1, num_training_steps - num_warmup_steps)
        )
        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress)))
    
    return LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)


def get_constant_warmup_scheduler(
    optimizer: Optimizer,
    num_warmup_steps: int,
    last_epoch: int = -1,
) -> LambdaLR:
    """
    Linear warmup followed by constant learning rate.
    
    Args:
        optimizer: Optimizer to schedule
        num_warmup_steps: Number of warmup steps
        last_epoch: Last epoch for resuming
        
    Returns:
        LambdaLR scheduler
    """
    def lr_lambda(current_step: int) -> float:
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return 1.0
    
    return LambdaLR(optimizer, lr_lambda, last_epoch=last_epoch)


def get_scheduler(
    name: str,
    optimizer: Optimizer,
    num_warmup_steps: int,
    num_training_steps: int,
    last_epoch: int = -1,
) -> LambdaLR:
    """
    Factory function for learning rate schedulers.
    
    Args:
        name: Scheduler name ('linear', 'cosine', 'constant')
        optimizer: Optimizer to schedule
        num_warmup_steps: Number of warmup steps
        num_training_steps: Total training steps
        last_epoch: Last epoch for resuming
        
    Returns:
        Learning rate scheduler
    """
    schedulers = {
        "linear": get_linear_warmup_scheduler,
        "cosine": get_cosine_warmup_scheduler,
        "constant": get_constant_warmup_scheduler,
    }
    
    if name not in schedulers:
        raise ValueError(f"Unknown scheduler: {name}. Choose from {list(schedulers.keys())}")
    
    scheduler_fn = schedulers[name]
    
    if name == "constant":
        return scheduler_fn(optimizer, num_warmup_steps, last_epoch)
    else:
        return scheduler_fn(optimizer, num_warmup_steps, num_training_steps, last_epoch=last_epoch)


def get_num_warmup_steps(
    num_training_steps: int,
    warmup_ratio: float = 0.1,
    warmup_steps: Optional[int] = None,
) -> int:
    """
    Calculate number of warmup steps.
    
    Args:
        num_training_steps: Total training steps
        warmup_ratio: Fraction of steps for warmup
        warmup_steps: Explicit warmup steps (overrides ratio)
        
    Returns:
        Number of warmup steps
    """
    if warmup_steps is not None:
        return warmup_steps
    return int(num_training_steps * warmup_ratio)
'''

with open("/content/VLM_Thesis/src/training/schedulers.py", 'w') as f:
    f.write(schedulers_py_content)
print("✅ Created: src/training/schedulers.py")

## Section 13: Trainer Implementation

The main training loop with Accelerate integration, mixed precision, gradient accumulation, checkpointing, and comprehensive logging.

In [ ]:
# ============================================================================
# FILE: src/training/trainer.py
# Main training loop with Accelerate
# ============================================================================

trainer_py_content = '''"""
VQA Trainer with Accelerate integration.

Handles training loop, validation, checkpointing, and logging.
"""

import os
import time
from typing import Dict, Any, Optional, Tuple
from tqdm.auto import tqdm

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import AdamW

from accelerate import Accelerator
from accelerate.utils import set_seed as accelerate_set_seed

# Local imports
import sys
sys.path.insert(0, '/content/VLM_Thesis')

from src.training.schedulers import get_scheduler, get_num_warmup_steps
from src.training.metrics import MetricTracker, VQAEvaluator, compute_vqa_metrics
from src.utils.io import save_checkpoint, load_checkpoint, save_json, ensure_dir
from src.utils.logging import ExperimentLogger, format_metrics, get_gpu_memory_info


class VQATrainer:
    """
    Trainer for VQA models.
    
    Features:
    - Accelerate for device management and mixed precision
    - Gradient accumulation and clipping
    - Checkpoint saving and resuming
    - TensorBoard/W&B logging
    - Early stopping
    - Smoke test mode
    """
    
    def __init__(
        self,
        model: nn.Module,
        train_loader: DataLoader,
        val_loader: DataLoader,
        config,
        processor=None,
    ):
        """
        Initialize trainer.
        
        Args:
            model: VQA model to train
            train_loader: Training data loader
            val_loader: Validation data loader
            config: Training configuration
            processor: BLIP-2 processor for decoding
        """
        self.config = config
        self.processor = processor
        
        # Initialize Accelerator
        self.accelerator = Accelerator(
            gradient_accumulation_steps=config.training.gradient_accumulation_steps,
            mixed_precision='fp16' if config.training.fp16 else 'no',
            log_with="tensorboard" if config.logging.use_tensorboard else None,
        )
        
        # Set seed
        accelerate_set_seed(config.seed)
        
        # Setup output directory
        self.output_dir = ensure_dir(os.path.join(
            config.logging.output_dir,
            config.logging.experiment_name
        ))
        self.checkpoint_dir = ensure_dir(os.path.join(self.output_dir, "checkpoints"))
        
        # Initialize optimizer
        self.optimizer = self._create_optimizer(model)
        
        # Calculate training steps
        self.num_update_steps_per_epoch = len(train_loader) // config.training.gradient_accumulation_steps
        
        if config.training.max_steps:
            self.max_steps = config.training.max_steps
            self.num_epochs = (self.max_steps // self.num_update_steps_per_epoch) + 1
        else:
            self.num_epochs = config.training.num_epochs
            self.max_steps = self.num_epochs * self.num_update_steps_per_epoch
        
        # Initialize scheduler
        num_warmup_steps = get_num_warmup_steps(
            self.max_steps,
            warmup_ratio=config.training.warmup_ratio
        )
        self.scheduler = get_scheduler(
            name=config.training.lr_scheduler_type,
            optimizer=self.optimizer,
            num_warmup_steps=num_warmup_steps,
            num_training_steps=self.max_steps,
        )
        
        # Prepare with Accelerator
        self.model, self.optimizer, self.train_loader, self.val_loader, self.scheduler = \
            self.accelerator.prepare(
                model, self.optimizer, train_loader, val_loader, self.scheduler
            )
        
        # Initialize logger
        self.logger = ExperimentLogger(
            log_dir=self.output_dir,
            experiment_name=config.logging.experiment_name,
            use_tensorboard=config.logging.use_tensorboard,
            use_wandb=config.logging.use_wandb,
            wandb_project=config.logging.wandb_project,
            config=config.to_dict(),
        )
        
        # Training state
        self.global_step = 0
        self.current_epoch = 0
        self.best_metric = 0.0
        self.early_stopping_counter = 0
        
        # Metric trackers
        self.train_metrics = MetricTracker(['loss', 'lr'])
        self.evaluator = VQAEvaluator()
        
        if self.accelerator.is_main_process:
            print(f"\\n🚀 Trainer initialized:")
            print(f"   Output dir: {self.output_dir}")
            print(f"   Device: {self.accelerator.device}")
            print(f"   Mixed precision: {self.accelerator.mixed_precision}")
            print(f"   Gradient accumulation: {config.training.gradient_accumulation_steps}")
            print(f"   Total epochs: {self.num_epochs}")
            print(f"   Max steps: {self.max_steps}")
            print(f"   Warmup steps: {num_warmup_steps}")
    
    def _create_optimizer(self, model: nn.Module) -> AdamW:
        """Create optimizer with proper parameter groups."""
        # Separate parameters that should and shouldn't have weight decay
        decay_params = []
        no_decay_params = []
        
        for name, param in model.named_parameters():
            if not param.requires_grad:
                continue
            if 'bias' in name or 'LayerNorm' in name or 'layernorm' in name:
                no_decay_params.append(param)
            else:
                decay_params.append(param)
        
        param_groups = [
            {'params': decay_params, 'weight_decay': self.config.training.weight_decay},
            {'params': no_decay_params, 'weight_decay': 0.0},
        ]
        
        return AdamW(param_groups, lr=self.config.training.learning_rate)
    
    def train(self) -> Dict[str, float]:
        """
        Run training loop.
        
        Returns:
            Final metrics dictionary
        """
        if self.accelerator.is_main_process:
            print(f"\\n{'='*60}")
            print("Starting Training")
            print(f"{'='*60}")
        
        # Save config
        self.config.save(os.path.join(self.output_dir, "config.yaml"))
        
        for epoch in range(self.num_epochs):
            self.current_epoch = epoch
            
            # Training epoch
            train_metrics = self._train_epoch()
            
            # Validation
            val_metrics = self._validate()
            
            # Log metrics
            if self.accelerator.is_main_process:
                print(f"\\nEpoch {epoch + 1}/{self.num_epochs}")
                print(f"  Train: {format_metrics(train_metrics)}")
                print(f"  Val:   {format_metrics(val_metrics)}")
                
                self.logger.log_metrics(train_metrics, self.global_step, prefix="train")
                self.logger.log_metrics(val_metrics, self.global_step, prefix="val")
            
            # Checkpoint
            is_best = val_metrics.get('normalized_match', 0) > self.best_metric
            if is_best:
                self.best_metric = val_metrics.get('normalized_match', 0)
                self.early_stopping_counter = 0
            else:
                self.early_stopping_counter += 1
            
            self._save_checkpoint(is_best=is_best)
            
            # Early stopping
            if self.config.training.early_stopping:
                if self.early_stopping_counter >= self.config.training.early_stopping_patience:
                    print(f"\\n⏹️ Early stopping at epoch {epoch + 1}")
                    break
            
            # Max steps check
            if self.global_step >= self.max_steps:
                print(f"\\n⏹️ Reached max steps: {self.max_steps}")
                break
        
        # Final save
        final_metrics = self._validate()
        self._save_checkpoint(is_best=False, filename="final.pt")
        
        self.logger.close()
        
        return final_metrics
    
    def _train_epoch(self) -> Dict[str, float]:
        """Run one training epoch."""
        self.model.train()
        self.train_metrics.reset()
        
        progress = tqdm(
            self.train_loader,
            desc=f"Epoch {self.current_epoch + 1}",
            disable=not self.accelerator.is_main_process,
        )
        
        for step, batch in enumerate(progress):
            # Forward pass with gradient accumulation
            with self.accelerator.accumulate(self.model):
                outputs = self.model(
                    pixel_values=batch['pixel_values'],
                    input_ids=batch['input_ids'],
                    attention_mask=batch['attention_mask'],
                    labels=batch['labels'],
                )
                
                loss = outputs['loss'] if isinstance(outputs, dict) else outputs.loss
                
                # Backward
                self.accelerator.backward(loss)
                
                # Gradient clipping
                if self.config.training.max_grad_norm > 0:
                    self.accelerator.clip_grad_norm_(
                        self.model.parameters(),
                        self.config.training.max_grad_norm
                    )
                
                # Optimizer step
                self.optimizer.step()
                self.scheduler.step()
                self.optimizer.zero_grad()
            
            # Update metrics
            self.train_metrics.update({
                'loss': loss.item(),
                'lr': self.scheduler.get_last_lr()[0],
            })
            
            # Logging
            if self.global_step % self.config.logging.log_every_n_steps == 0:
                progress.set_postfix({
                    'loss': f"{self.train_metrics.metrics['loss'].avg:.4f}",
                    'lr': f"{self.scheduler.get_last_lr()[0]:.2e}",
                })
                
                if self.accelerator.is_main_process:
                    self.logger.log_scalar('train/loss', loss.item(), self.global_step)
                    self.logger.log_learning_rate(
                        self.scheduler.get_last_lr()[0], self.global_step
                    )
                    self.logger.log_gpu_memory(self.global_step)
            
            self.global_step += 1
            
            # Max steps check
            if self.global_step >= self.max_steps:
                break
        
        return self.train_metrics.get_averages()
    
    @torch.no_grad()
    def _validate(self) -> Dict[str, float]:
        """Run validation."""
        self.model.eval()
        self.evaluator.reset()
        
        val_loss = 0.0
        num_batches = 0
        
        progress = tqdm(
            self.val_loader,
            desc="Validation",
            disable=not self.accelerator.is_main_process,
        )
        
        for batch in progress:
            # Get loss
            outputs = self.model(
                pixel_values=batch['pixel_values'],
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                labels=batch['labels'],
            )
            
            loss = outputs['loss'] if isinstance(outputs, dict) else outputs.loss
            val_loss += loss.item()
            num_batches += 1
            
            # Generate predictions
            unwrapped_model = self.accelerator.unwrap_model(self.model)
            predictions = unwrapped_model.generate(
                pixel_values=batch['pixel_values'],
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
            )
            
            # Add to evaluator
            self.evaluator.add_batch(
                predictions=predictions,
                targets=batch['answers'],
                question_ids=batch.get('question_ids'),
                questions=batch.get('questions'),
            )
        
        # Compute metrics
        metrics = self.evaluator.compute_metrics()
        metrics['loss'] = val_loss / max(num_batches, 1)
        
        return metrics
    
    def _save_checkpoint(self, is_best: bool = False, filename: Optional[str] = None):
        """Save training checkpoint."""
        if not self.accelerator.is_main_process:
            return
        
        if filename is None:
            filename = f"checkpoint_epoch{self.current_epoch + 1}.pt"
        
        checkpoint_path = os.path.join(self.checkpoint_dir, filename)
        
        state_dict = {
            'epoch': self.current_epoch,
            'global_step': self.global_step,
            'model_state_dict': self.accelerator.unwrap_model(self.model).state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_metric': self.best_metric,
            'config': self.config.to_dict(),
        }
        
        save_checkpoint(
            state_dict,
            checkpoint_path,
            is_best=is_best,
            keep_last_n=self.config.training.save_total_limit,
        )
    
    def resume_from_checkpoint(self, checkpoint_path: str):
        """Resume training from checkpoint."""
        checkpoint = load_checkpoint(checkpoint_path)
        
        self.accelerator.unwrap_model(self.model).load_state_dict(
            checkpoint['model_state_dict']
        )
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        
        self.current_epoch = checkpoint['epoch']
        self.global_step = checkpoint['global_step']
        self.best_metric = checkpoint.get('best_metric', 0.0)
        
        print(f"✅ Resumed from epoch {self.current_epoch}, step {self.global_step}")
'''

with open("/content/VLM_Thesis/src/training/trainer.py", 'w') as f:
    f.write(trainer_py_content)
print("✅ Created: src/training/trainer.py")

## Section 14: Evaluation Script

Standalone evaluation module for model assessment and metric computation.

In [ ]:
# ============================================================================
# FILE: src/evaluation/evaluate.py
# Evaluation script for VQA models
# ============================================================================

evaluate_py_content = '''"""
Evaluation module for VQA models.

Loads checkpoints and evaluates on validation/test sets.
"""

import os
import sys
from typing import Dict, Any, Optional, List
from tqdm.auto import tqdm

import torch
from torch.utils.data import DataLoader

sys.path.insert(0, '/content/VLM_Thesis')

from src.training.metrics import VQAEvaluator, compute_vqa_metrics
from src.utils.io import load_checkpoint, save_json, save_csv


class VQAEvaluatorPipeline:
    """
    Complete evaluation pipeline for VQA models.
    """
    
    def __init__(
        self,
        model,
        dataloader: DataLoader,
        processor=None,
        device: str = 'cuda' if torch.cuda.is_available() else 'cpu',
    ):
        """
        Initialize evaluator.
        
        Args:
            model: VQA model (loaded with weights)
            dataloader: Evaluation data loader
            processor: BLIP-2 processor for decoding
            device: Device for evaluation
        """
        self.model = model.to(device)
        self.dataloader = dataloader
        self.processor = processor
        self.device = device
        
        self.model.eval()
    
    @torch.no_grad()
    def evaluate(
        self,
        save_predictions: bool = True,
        output_dir: Optional[str] = None,
    ) -> Dict[str, Any]:
        """
        Run evaluation.
        
        Args:
            save_predictions: Whether to save individual predictions
            output_dir: Directory for saving outputs
            
        Returns:
            Dictionary with metrics and optional predictions
        """
        evaluator = VQAEvaluator()
        
        all_predictions = []
        
        progress = tqdm(self.dataloader, desc="Evaluating")
        
        for batch in progress:
            # Move to device
            pixel_values = batch['pixel_values'].to(self.device)
            input_ids = batch['input_ids'].to(self.device)
            attention_mask = batch['attention_mask'].to(self.device)
            
            # Generate predictions
            predictions = self.model.generate(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
            )
            
            # Collect results
            evaluator.add_batch(
                predictions=predictions,
                targets=batch['answers'],
                question_ids=batch.get('question_ids'),
                questions=batch.get('questions'),
            )
            
            # Store individual predictions
            for i, (pred, target) in enumerate(zip(predictions, batch['answers'])):
                all_predictions.append({
                    'question_id': batch['question_ids'][i] if 'question_ids' in batch else i,
                    'question': batch['questions'][i] if 'questions' in batch else '',
                    'prediction': pred,
                    'target': target,
                })
        
        # Compute metrics
        metrics = evaluator.compute_metrics()
        
        # Get detailed results
        detailed_results = evaluator.get_results_df()
        
        # Save if requested
        if save_predictions and output_dir:
            os.makedirs(output_dir, exist_ok=True)
            
            # Save metrics
            save_json(metrics, os.path.join(output_dir, 'metrics.json'))
            
            # Save predictions
            save_csv(all_predictions, os.path.join(output_dir, 'predictions.csv'))
            
            # Save detailed results
            save_json(detailed_results, os.path.join(output_dir, 'detailed_results.json'))
        
        return {
            'metrics': metrics,
            'predictions': all_predictions,
            'detailed_results': detailed_results,
        }


def evaluate_checkpoint(
    checkpoint_path: str,
    config_path: str,
    split: str = 'validation',
    output_dir: Optional[str] = None,
) -> Dict[str, float]:
    """
    Evaluate a saved checkpoint.
    
    Args:
        checkpoint_path: Path to model checkpoint
        config_path: Path to config YAML
        split: Dataset split to evaluate
        output_dir: Directory for outputs
        
    Returns:
        Evaluation metrics
    """
    from src.utils.config import Config
    from src.models.blip2_wrapper import create_blip2_model
    from src.datasets.vqa_dataset import VQADataset, vqa_collate_fn
    
    # Load config
    config = Config.from_yaml(config_path)
    
    # Create model
    model = create_blip2_model(config)
    
    # Load checkpoint
    checkpoint = load_checkpoint(checkpoint_path)
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    
    # Create dataset
    dataset = VQADataset(
        processor=model.get_processor(),
        split=split,
        dataset_name=config.data.dataset_name,
        max_samples=config.data.max_samples_val,
        cache_dir=config.data.cache_dir,
    )
    
    dataloader = DataLoader(
        dataset,
        batch_size=config.training.batch_size,
        shuffle=False,
        num_workers=config.data.num_workers,
        collate_fn=vqa_collate_fn,
    )
    
    # Evaluate
    evaluator = VQAEvaluatorPipeline(
        model=model,
        dataloader=dataloader,
        processor=model.get_processor(),
    )
    
    results = evaluator.evaluate(
        save_predictions=True,
        output_dir=output_dir or config.logging.output_dir,
    )
    
    return results['metrics']


def print_evaluation_summary(metrics: Dict[str, float]) -> None:
    """Print formatted evaluation summary."""
    print("\\n" + "="*50)
    print("EVALUATION RESULTS")
    print("="*50)
    
    for key, value in metrics.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")
    
    print("="*50)
'''

with open("/content/VLM_Thesis/src/evaluation/evaluate.py", 'w') as f:
    f.write(evaluate_py_content)
print("✅ Created: src/evaluation/evaluate.py")

## Section 15: Error Analysis Module

Tools for analyzing model errors and identifying patterns in incorrect predictions.

In [ ]:
# ============================================================================
# FILE: src/evaluation/error_analysis.py
# Error analysis utilities for VQA
# ============================================================================

error_analysis_py_content = '''"""
Error Analysis module for VQA evaluation.

Provides tools for analyzing prediction errors and identifying patterns.
"""

import os
from typing import Dict, List, Any, Optional
from collections import Counter
import json

import sys
sys.path.insert(0, '/content/VLM_Thesis')

from src.datasets.answer_vocab import normalize_answer, normalized_match
from src.utils.io import save_json, save_csv


class ErrorAnalyzer:
    """
    Analyzes VQA model errors to identify patterns and common mistakes.
    """
    
    def __init__(self, predictions: List[Dict[str, Any]]):
        """
        Initialize error analyzer.
        
        Args:
            predictions: List of prediction dictionaries with keys:
                - question_id
                - question
                - prediction
                - target
        """
        self.predictions = predictions
        self.errors = []
        self.correct = []
        
        self._classify_predictions()
    
    def _classify_predictions(self) -> None:
        """Classify predictions as correct or incorrect."""
        for pred in self.predictions:
            is_correct = normalized_match(pred['prediction'], pred['target'])
            pred['is_correct'] = is_correct
            pred['normalized_prediction'] = normalize_answer(pred['prediction'])
            pred['normalized_target'] = normalize_answer(pred['target'])
            
            if is_correct:
                self.correct.append(pred)
            else:
                self.errors.append(pred)
    
    def get_error_rate(self) -> float:
        """Get overall error rate."""
        if not self.predictions:
            return 0.0
        return len(self.errors) / len(self.predictions)
    
    def get_accuracy(self) -> float:
        """Get overall accuracy."""
        return 1.0 - self.get_error_rate()
    
    def get_most_common_wrong_predictions(
        self,
        n: int = 10
    ) -> List[Dict[str, Any]]:
        """
        Get the most common incorrect predictions.
        
        Args:
            n: Number of top errors to return
            
        Returns:
            List of (prediction, count, examples) tuples
        """
        # Count wrong predictions
        wrong_pred_counter = Counter()
        wrong_examples = {}
        
        for error in self.errors:
            pred = error['normalized_prediction']
            wrong_pred_counter[pred] += 1
            
            if pred not in wrong_examples:
                wrong_examples[pred] = []
            if len(wrong_examples[pred]) < 3:  # Store up to 3 examples
                wrong_examples[pred].append({
                    'question': error['question'],
                    'target': error['target'],
                })
        
        # Format results
        results = []
        for pred, count in wrong_pred_counter.most_common(n):
            results.append({
                'wrong_prediction': pred,
                'count': count,
                'examples': wrong_examples.get(pred, []),
            })
        
        return results
    
    def get_confusion_pairs(
        self,
        n: int = 10
    ) -> List[Dict[str, Any]]:
        """
        Get most common (prediction, target) confusion pairs.
        
        Args:
            n: Number of top pairs to return
            
        Returns:
            List of confusion pair statistics
        """
        pair_counter = Counter()
        
        for error in self.errors:
            pred = error['normalized_prediction']
            target = error['normalized_target']
            pair_counter[(pred, target)] += 1
        
        results = []
        for (pred, target), count in pair_counter.most_common(n):
            results.append({
                'prediction': pred,
                'target': target,
                'count': count,
            })
        
        return results
    
    def get_error_by_answer_type(self) -> Dict[str, Dict[str, float]]:
        """
        Analyze errors by answer type (yes/no, number, other).
        
        Returns:
            Dictionary of error rates by type
        """
        type_stats = {
            'yes_no': {'correct': 0, 'total': 0},
            'number': {'correct': 0, 'total': 0},
            'other': {'correct': 0, 'total': 0},
        }
        
        yes_no_answers = {'yes', 'no'}
        
        for pred in self.predictions:
            target = pred['normalized_target']
            
            # Classify type
            if target in yes_no_answers:
                answer_type = 'yes_no'
            elif target.isdigit():
                answer_type = 'number'
            else:
                answer_type = 'other'
            
            type_stats[answer_type]['total'] += 1
            if pred['is_correct']:
                type_stats[answer_type]['correct'] += 1
        
        # Compute accuracy per type
        results = {}
        for atype, stats in type_stats.items():
            if stats['total'] > 0:
                results[atype] = {
                    'accuracy': stats['correct'] / stats['total'],
                    'total': stats['total'],
                    'correct': stats['correct'],
                }
        
        return results
    
    def save_error_report(self, output_dir: str) -> None:
        """
        Save comprehensive error analysis report.
        
        Args:
            output_dir: Directory to save reports
        """
        os.makedirs(output_dir, exist_ok=True)
        
        # Save all errors
        save_csv(
            self.errors,
            os.path.join(output_dir, 'errors.csv'),
            fieldnames=['question_id', 'question', 'prediction', 'target', 
                       'normalized_prediction', 'normalized_target']
        )
        
        # Save error summary
        summary = {
            'total_predictions': len(self.predictions),
            'total_correct': len(self.correct),
            'total_errors': len(self.errors),
            'accuracy': self.get_accuracy(),
            'error_rate': self.get_error_rate(),
            'most_common_wrong': self.get_most_common_wrong_predictions(10),
            'confusion_pairs': self.get_confusion_pairs(10),
            'error_by_type': self.get_error_by_answer_type(),
        }
        
        save_json(summary, os.path.join(output_dir, 'error_summary.json'))
        
        print(f"📊 Error report saved to {output_dir}")
    
    def print_summary(self) -> None:
        """Print error analysis summary."""
        print("\\n" + "="*50)
        print("ERROR ANALYSIS SUMMARY")
        print("="*50)
        
        print(f"\\nOverall Statistics:")
        print(f"  Total: {len(self.predictions)}")
        print(f"  Correct: {len(self.correct)} ({self.get_accuracy()*100:.1f}%)")
        print(f"  Errors: {len(self.errors)} ({self.get_error_rate()*100:.1f}%)")
        
        print(f"\\nMost Common Wrong Predictions:")
        for item in self.get_most_common_wrong_predictions(5):
            print(f"  '{item['wrong_prediction']}': {item['count']} times")
        
        print(f"\\nError by Answer Type:")
        for atype, stats in self.get_error_by_answer_type().items():
            print(f"  {atype}: {stats['accuracy']*100:.1f}% ({stats['correct']}/{stats['total']})")
        
        print("="*50)


def analyze_attention_weights(
    attention_weights: Dict[str, Any],
    output_dir: str,
    sample_id: str = "sample",
) -> None:
    """
    Save attention weight visualizations.
    
    Args:
        attention_weights: Dictionary of attention matrices
        output_dir: Directory to save visualizations
        sample_id: Sample identifier
    """
    import matplotlib.pyplot as plt
    import numpy as np
    
    os.makedirs(output_dir, exist_ok=True)
    
    for layer_name, weights in attention_weights.items():
        if isinstance(weights, dict):
            continue
            
        # Convert to numpy
        if hasattr(weights, 'cpu'):
            weights = weights.cpu().numpy()
        
        # Handle batched attention
        if weights.ndim == 3:
            weights = weights[0]  # Take first sample
        
        # Plot heatmap
        plt.figure(figsize=(10, 10))
        plt.imshow(weights, cmap='viridis', aspect='auto')
        plt.colorbar(label='Attention Weight')
        plt.title(f'Attention: {layer_name}')
        plt.xlabel('Key Position')
        plt.ylabel('Query Position')
        
        # Save
        save_path = os.path.join(output_dir, f'{sample_id}_{layer_name}.png')
        plt.savefig(save_path, dpi=150, bbox_inches='tight')
        plt.close()
    
    print(f"📊 Attention visualizations saved to {output_dir}")
'''

with open("/content/VLM_Thesis/src/evaluation/error_analysis.py", 'w') as f:
    f.write(error_analysis_py_content)
print("✅ Created: src/evaluation/error_analysis.py")

## Section 16: Training Script

Main entry point for training with CLI argument parsing.

In [ ]:
# ============================================================================
# FILE: scripts/train.py
# Main training script with execution profile support
# ============================================================================

train_script_content = '''#!/usr/bin/env python3
"""
VQA Training Script

Supports multiple execution profiles:
    - colab_train: Full training on Colab GPU (default)
    - mac_dev: Local development (smoke/sanity runs only)
    - eval_only: Evaluation mode (no training)

Usage:
    # Colab - Full training
    python scripts/train.py --config configs/baseline.yaml
    python scripts/train.py --config configs/proposed.yaml --sync_to_drive
    
    # Mac - Development (auto-limited)
    python scripts/train.py --config configs/baseline_mac.yaml --execution_profile mac_dev
    
    # Smoke test (any environment)
    python scripts/train.py --config configs/baseline.yaml --smoke_test true
"""

import os
import sys
import shutil

# Add project root to path
sys.path.insert(0, '/content/VLM_Thesis')

from src.utils.config import get_argument_parser, load_config, detect_environment
from src.utils.seed import set_seed
from src.models.blip2_wrapper import create_blip2_model
from src.datasets.vqa_dataset import create_dataloaders
from src.training.trainer import VQATrainer


def setup_drive_sync(config):
    """Setup Google Drive sync if enabled."""
    if not config.runtime.sync_to_drive:
        return None
    
    try:
        from google.colab import drive
        
        # Mount Drive if not already mounted
        if not os.path.exists('/content/drive'):
            print("📁 Mounting Google Drive...")
            drive.mount('/content/drive')
        
        # Create sync directory
        sync_path = config.runtime.drive_mount_path
        os.makedirs(sync_path, exist_ok=True)
        print(f"✅ Drive sync enabled: {sync_path}")
        
        return sync_path
        
    except ImportError:
        print("⚠️ Google Drive sync only available in Colab")
        return None


def sync_outputs_to_drive(config, drive_path):
    """Sync outputs to Google Drive."""
    if drive_path is None:
        return
    
    output_dir = config.logging.output_dir
    experiment_name = config.logging.experiment_name
    
    # Copy outputs to Drive
    src = os.path.join(output_dir, experiment_name)
    dst = os.path.join(drive_path, experiment_name)
    
    if os.path.exists(src):
        print(f"📤 Syncing to Drive: {dst}")
        if os.path.exists(dst):
            shutil.rmtree(dst)
        shutil.copytree(src, dst)
        print(f"✅ Sync complete")


def main():
    """Main training function."""
    
    # Parse arguments
    parser = get_argument_parser()
    args = parser.parse_args()
    
    # Auto-detect environment if profile not specified
    if args.execution_profile is None:
        args.execution_profile = detect_environment()
        print(f"🔍 Auto-detected environment: {args.execution_profile}")
    
    # Load configuration
    print(f"📋 Loading config: {args.config}")
    config = load_config(args.config, args)
    
    # Print profile info
    config.print_profile_info()
    
    # Check if training is allowed
    if not config.runtime.is_training_allowed():
        print("❌ Training not allowed in eval_only profile.")
        print("   Use --execution_profile colab_train for training.")
        return None
    
    # Setup Drive sync
    drive_path = setup_drive_sync(config)
    
    # Set seed for reproducibility
    set_seed(config.seed)
    
    # Create model
    print(f"\\n🤖 Creating model...")
    model = create_blip2_model(config)
    
    # Create dataloaders
    print(f"\\n📚 Creating dataloaders...")
    train_loader, val_loader = create_dataloaders(
        processor=model.get_processor(),
        config=config,
        seed=config.seed,
    )
    
    print(f"   Train samples: {len(train_loader.dataset)}")
    print(f"   Val samples: {len(val_loader.dataset)}")
    
    # Create trainer
    trainer = VQATrainer(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        config=config,
        processor=model.get_processor(),
    )
    
    # Resume if checkpoint provided
    if args.ckpt:
        trainer.resume_from_checkpoint(args.ckpt)
    
    # Train
    final_metrics = trainer.train()
    
    # Sync to Drive if enabled
    sync_outputs_to_drive(config, drive_path)
    
    # Print final results
    print(f"\\n{'='*60}")
    print("TRAINING COMPLETE")
    print(f"{'='*60}")
    for key, value in final_metrics.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.4f}")
        else:
            print(f"  {key}: {value}")
    
    return final_metrics


if __name__ == "__main__":
    main()
'''

with open("/content/VLM_Thesis/scripts/train.py", 'w') as f:
    f.write(train_script_content)
print("✅ Created: scripts/train.py")

# ============================================================================
# FILE: scripts/eval.py  
# Evaluation script
# ============================================================================

eval_script_content = '''#!/usr/bin/env python3
"""
VQA Evaluation Script

Usage:
    python scripts/eval.py --config configs/baseline.yaml --ckpt outputs/checkpoints/best.pt
"""

import os
import sys
import argparse

sys.path.insert(0, '/content/VLM_Thesis')

from src.utils.config import load_config
from src.utils.seed import set_seed
from src.models.blip2_wrapper import create_blip2_model
from src.datasets.vqa_dataset import VQADataset, vqa_collate_fn
from src.evaluation.evaluate import VQAEvaluatorPipeline, print_evaluation_summary
from src.evaluation.error_analysis import ErrorAnalyzer
from src.utils.io import load_checkpoint, save_json

import torch
from torch.utils.data import DataLoader


def main():
    """Main evaluation function."""
    
    parser = argparse.ArgumentParser(description="VQA Evaluation")
    parser.add_argument("--config", type=str, required=True, help="Config file path")
    parser.add_argument("--ckpt", type=str, required=True, help="Checkpoint path")
    parser.add_argument("--split", type=str, default="validation", help="Dataset split")
    parser.add_argument("--output_dir", type=str, default=None, help="Output directory")
    parser.add_argument("--batch_size", type=int, default=None, help="Batch size")
    args = parser.parse_args()
    
    # Load config
    print(f"📋 Loading config: {args.config}")
    config = load_config(args.config)
    
    if args.batch_size:
        config.training.batch_size = args.batch_size
    
    # Set seed
    set_seed(config.seed)
    
    # Create model
    print(f"\\n🤖 Creating model...")
    model = create_blip2_model(config)
    
    # Load checkpoint
    print(f"\\n📂 Loading checkpoint: {args.ckpt}")
    checkpoint = load_checkpoint(args.ckpt)
    model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    
    # Create dataset and loader
    print(f"\\n📚 Creating dataset ({args.split})...")
    dataset = VQADataset(
        processor=model.get_processor(),
        split=args.split,
        dataset_name=config.data.dataset_name,
        max_samples=config.data.max_samples_val,
        cache_dir=config.data.cache_dir,
    )
    
    dataloader = DataLoader(
        dataset,
        batch_size=config.training.batch_size,
        shuffle=False,
        num_workers=config.data.num_workers,
        collate_fn=vqa_collate_fn,
    )
    
    print(f"   Samples: {len(dataset)}")
    
    # Setup output directory
    output_dir = args.output_dir or os.path.join(config.logging.output_dir, "evaluation")
    os.makedirs(output_dir, exist_ok=True)
    
    # Evaluate
    print(f"\\n🔍 Running evaluation...")
    evaluator = VQAEvaluatorPipeline(
        model=model,
        dataloader=dataloader,
        processor=model.get_processor(),
    )
    
    results = evaluator.evaluate(
        save_predictions=True,
        output_dir=output_dir,
    )
    
    # Print summary
    print_evaluation_summary(results['metrics'])
    
    # Error analysis
    print(f"\\n📊 Running error analysis...")
    analyzer = ErrorAnalyzer(results['predictions'])
    analyzer.save_error_report(os.path.join(output_dir, "error_analysis"))
    analyzer.print_summary()
    
    print(f"\\n✅ Results saved to: {output_dir}")
    
    return results['metrics']


if __name__ == "__main__":
    main()
'''

with open("/content/VLM_Thesis/scripts/eval.py", 'w') as f:
    f.write(eval_script_content)
print("✅ Created: scripts/eval.py")

## Section 17: Report Generation Script

Script for aggregating results and generating thesis-ready reports.

In [ ]:
# ============================================================================
# FILE: scripts/make_report.py
# Report generation script
# ============================================================================

make_report_content = '''#!/usr/bin/env python3
"""
Report Generation Script

Aggregates results from multiple experiments and generates thesis-ready reports.

Usage:
    python scripts/make_report.py --results_dir outputs/
"""

import os
import sys
import argparse
import json
import glob
from datetime import datetime
from typing import Dict, List, Any

sys.path.insert(0, '/content/VLM_Thesis')

from src.utils.io import load_json, save_json


def find_experiment_results(results_dir: str) -> List[Dict[str, Any]]:
    """
    Find all experiment results in directory.
    
    Args:
        results_dir: Directory containing experiment outputs
        
    Returns:
        List of experiment result dictionaries
    """
    experiments = []
    
    # Look for metrics.json files
    for metrics_path in glob.glob(os.path.join(results_dir, "**/metrics.json"), recursive=True):
        try:
            metrics = load_json(metrics_path)
            
            # Get experiment name from path
            exp_dir = os.path.dirname(metrics_path)
            exp_name = os.path.basename(exp_dir)
            
            # Try to load config
            config_path = os.path.join(exp_dir, "config.yaml")
            config = None
            if os.path.exists(config_path):
                import yaml
                with open(config_path) as f:
                    config = yaml.safe_load(f)
            
            experiments.append({
                'name': exp_name,
                'metrics': metrics,
                'config': config,
                'path': exp_dir,
            })
            
        except Exception as e:
            print(f"Warning: Could not load {metrics_path}: {e}")
    
    return experiments


def generate_comparison_table(experiments: List[Dict[str, Any]]) -> str:
    """
    Generate Markdown comparison table.
    
    Args:
        experiments: List of experiment results
        
    Returns:
        Markdown table string
    """
    if not experiments:
        return "No experiments found."
    
    # Get all metric keys
    all_metrics = set()
    for exp in experiments:
        all_metrics.update(exp['metrics'].keys())
    
    # Filter to numeric metrics
    numeric_metrics = []
    for metric in sorted(all_metrics):
        sample_val = experiments[0]['metrics'].get(metric)
        if isinstance(sample_val, (int, float)):
            numeric_metrics.append(metric)
    
    # Build table
    lines = []
    
    # Header
    header = "| Experiment | " + " | ".join(numeric_metrics) + " |"
    separator = "|" + "|".join(["---"] * (len(numeric_metrics) + 1)) + "|"
    lines.append(header)
    lines.append(separator)
    
    # Rows
    for exp in experiments:
        row = f"| {exp['name']} |"
        for metric in numeric_metrics:
            val = exp['metrics'].get(metric, 'N/A')
            if isinstance(val, float):
                row += f" {val:.4f} |"
            else:
                row += f" {val} |"
        lines.append(row)
    
    return "\\n".join(lines)


def generate_results_summary(experiments: List[Dict[str, Any]]) -> str:
    """
    Generate results summary for thesis.
    
    Args:
        experiments: List of experiment results
        
    Returns:
        Markdown summary string
    """
    lines = [
        "# Experimental Results Summary",
        "",
        f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "",
        "## Overview",
        "",
        f"Total experiments: {len(experiments)}",
        "",
    ]
    
    if experiments:
        # Find best performing model
        best_exp = max(experiments, 
                       key=lambda x: x['metrics'].get('normalized_match', 0))
        lines.extend([
            "## Best Performing Model",
            "",
            f"**{best_exp['name']}**",
            "",
        ])
        
        for metric, value in best_exp['metrics'].items():
            if isinstance(value, float):
                lines.append(f"- {metric}: {value:.4f}")
            else:
                lines.append(f"- {metric}: {value}")
        
        lines.extend([
            "",
            "## Comparison Table",
            "",
            generate_comparison_table(experiments),
            "",
        ])
    
    # Add analysis section template
    lines.extend([
        "## Analysis",
        "",
        "### Key Findings",
        "",
        "1. [Finding 1]",
        "2. [Finding 2]",
        "3. [Finding 3]",
        "",
        "### Scene Reasoning Module Impact",
        "",
        "[Analysis of scene reasoning module contribution]",
        "",
        "### Ablation Study Results",
        "",
        "[Discussion of ablation experiments]",
        "",
    ])
    
    return "\\n".join(lines)


def save_csv_report(experiments: List[Dict[str, Any]], output_path: str) -> None:
    """Save results as CSV."""
    import csv
    
    if not experiments:
        return
    
    # Get all metric keys
    all_metrics = set()
    for exp in experiments:
        all_metrics.update(exp['metrics'].keys())
    
    fieldnames = ['experiment'] + sorted(list(all_metrics))
    
    with open(output_path, 'w', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        
        for exp in experiments:
            row = {'experiment': exp['name']}
            row.update(exp['metrics'])
            writer.writerow(row)
    
    print(f"📊 CSV report saved: {output_path}")


def main():
    """Main report generation function."""
    
    parser = argparse.ArgumentParser(description="Generate experiment reports")
    parser.add_argument("--results_dir", type=str, default="/content/VLM_Thesis/outputs",
                        help="Directory containing experiment results")
    parser.add_argument("--output_dir", type=str, default="/content/VLM_Thesis/thesis_assets",
                        help="Output directory for reports")
    args = parser.parse_args()
    
    print(f"📂 Scanning for results in: {args.results_dir}")
    
    # Find experiments
    experiments = find_experiment_results(args.results_dir)
    print(f"   Found {len(experiments)} experiments")
    
    # Create output directory
    os.makedirs(args.output_dir, exist_ok=True)
    
    # Generate comparison table
    table_md = generate_comparison_table(experiments)
    table_path = os.path.join(args.output_dir, "experiment_table.md")
    with open(table_path, 'w') as f:
        f.write("# Experiment Comparison\\n\\n")
        f.write(table_md)
    print(f"📋 Table saved: {table_path}")
    
    # Generate results summary
    summary = generate_results_summary(experiments)
    summary_path = os.path.join(args.output_dir, "results_summary.md")
    with open(summary_path, 'w') as f:
        f.write(summary)
    print(f"📋 Summary saved: {summary_path}")
    
    # Save CSV
    csv_path = os.path.join(args.output_dir, "experiment_results.csv")
    save_csv_report(experiments, csv_path)
    
    print(f"\\n✅ Reports generated in: {args.output_dir}")


if __name__ == "__main__":
    main()
'''

with open("/content/VLM_Thesis/scripts/make_report.py", 'w') as f:
    f.write(make_report_content)
print("✅ Created: scripts/make_report.py")

## Section 18: Configuration Files

YAML configuration files for all experiments.

In [ ]:
# ============================================================================
# CONFIG: baseline.yaml - BLIP-2 baseline (no scene reasoning)
# ============================================================================

baseline_config = '''# Baseline Configuration: BLIP-2 for VQA
# No scene reasoning module - pure BLIP-2 with generative VQA

experiment:
  name: "baseline_blip2"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/baseline"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: false
  max_length: 32

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"  # Colab-friendly subset
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129  # Standard VQA answer vocabulary size

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/baseline.yaml", 'w') as f:
    f.write(baseline_config)
print("✅ Created: configs/baseline.yaml")

# ============================================================================
# CONFIG: proposed.yaml - BLIP-2 + Scene Reasoning (full model)
# ============================================================================

proposed_config = '''# Proposed Configuration: BLIP-2 + Scene Reasoning Module
# Full model with spatial encoding and relation-aware attention

experiment:
  name: "proposed_scene_reasoning"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/proposed"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: true
  max_length: 32
  
  scene_reasoning:
    hidden_dim: 768
    num_heads: 8
    num_layers: 2
    dropout: 0.1
    use_spatial_encoding: true
    use_relation_attention: true
    spatial_dim: 64
    max_objects: 100

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/proposed.yaml", 'w') as f:
    f.write(proposed_config)
print("✅ Created: configs/proposed.yaml")

# ============================================================================
# CONFIG: ablation_no_spatial.yaml - Without spatial encoding
# ============================================================================

ablation_no_spatial = '''# Ablation: No Spatial Encoding
# Scene reasoning with relation attention only

experiment:
  name: "ablation_no_spatial"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/ablation_no_spatial"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: true
  max_length: 32
  
  scene_reasoning:
    hidden_dim: 768
    num_heads: 8
    num_layers: 2
    dropout: 0.1
    use_spatial_encoding: false  # ABLATION: disabled
    use_relation_attention: true
    spatial_dim: 64
    max_objects: 100

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/ablation_no_spatial.yaml", 'w') as f:
    f.write(ablation_no_spatial)
print("✅ Created: configs/ablation_no_spatial.yaml")

# ============================================================================
# CONFIG: ablation_no_relation.yaml - Without relation attention
# ============================================================================

ablation_no_relation = '''# Ablation: No Relation Attention
# Scene reasoning with spatial encoding only

experiment:
  name: "ablation_no_relation"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/ablation_no_relation"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: true
  max_length: 32
  
  scene_reasoning:
    hidden_dim: 768
    num_heads: 8
    num_layers: 2
    dropout: 0.1
    use_spatial_encoding: true
    use_relation_attention: false  # ABLATION: disabled
    spatial_dim: 64
    max_objects: 100

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/ablation_no_relation.yaml", 'w') as f:
    f.write(ablation_no_relation)
print("✅ Created: configs/ablation_no_relation.yaml")

# ============================================================================
# CONFIG: ablation_no_reasoning.yaml - No scene reasoning at all
# ============================================================================

ablation_no_reasoning = '''# Ablation: No Scene Reasoning
# Equivalent to baseline - for comparison in ablation table

experiment:
  name: "ablation_no_reasoning"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/ablation_no_reasoning"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: false  # ABLATION: No scene reasoning
  max_length: 32

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/ablation_no_reasoning.yaml", 'w') as f:
    f.write(ablation_no_reasoning)
print("✅ Created: configs/ablation_no_reasoning.yaml")

# ============================================================================
# CONFIG OVERLAYS: Mac Development Configs
# ============================================================================

baseline_mac_config = '''# Baseline Configuration for Mac Development
# Use for local development and testing ONLY - not for actual training
# IMPORTANT: Model is UNCHANGED - same BLIP-2, just with dev-friendly settings

experiment:
  name: "baseline_mac_dev"
  seed: 42
  output_dir: "./outputs/baseline_mac"

model:
  # SAME MODEL - no reduction in size or capability
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: false
  max_length: 32

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:50]"   # Small subset for dev
  val_split: "validation[:25]"
  max_answer_length: 10
  top_k_answers: 3129
  cache_dir: "~/.cache/huggingface/datasets"

training:
  batch_size: 1
  gradient_accumulation_steps: 1
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 1
  max_steps: 5  # Safety limit for Mac
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: false   # MPS has limited fp16 support
  device: "mps"  # or "cpu"
  save_checkpoints: false  # Disable checkpoints for dev
  scheduler: "cosine"
  save_steps: 1000
  eval_steps: 1000
  logging_steps: 1

runtime:
  execution_profile: "mac_dev"
  mac_dev_max_steps: 10
  mac_dev_max_samples: 50
  mac_dev_allow_checkpoints: false

evaluation:
  batch_size: 1
  num_beams: 1  # Faster for dev
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/baseline_mac.yaml", 'w') as f:
    f.write(baseline_mac_config)
print("✅ Created: configs/baseline_mac.yaml")

proposed_mac_config = '''# Proposed Model Configuration for Mac Development
# Use for local development and testing ONLY - not for actual training
# IMPORTANT: Model is UNCHANGED - same BLIP-2 + Scene Reasoning

experiment:
  name: "proposed_mac_dev"
  seed: 42
  output_dir: "./outputs/proposed_mac"

model:
  # SAME MODEL - no reduction in size or capability
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: true
  max_length: 32
  
  scene_reasoning:
    hidden_dim: 768
    num_heads: 8
    num_layers: 2
    dropout: 0.1
    use_spatial_encoding: true
    use_relation_attention: true
    spatial_dim: 64
    max_objects: 100

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:50]"
  val_split: "validation[:25]"
  max_answer_length: 10
  top_k_answers: 3129
  cache_dir: "~/.cache/huggingface/datasets"

training:
  batch_size: 1
  gradient_accumulation_steps: 1
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 1
  max_steps: 5  # Safety limit for Mac
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: false
  device: "mps"
  save_checkpoints: false
  scheduler: "cosine"
  save_steps: 1000
  eval_steps: 1000
  logging_steps: 1

runtime:
  execution_profile: "mac_dev"
  mac_dev_max_steps: 10
  mac_dev_max_samples: 50
  mac_dev_allow_checkpoints: false

evaluation:
  batch_size: 1
  num_beams: 1
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/proposed_mac.yaml", 'w') as f:
    f.write(proposed_mac_config)
print("✅ Created: configs/proposed_mac.yaml")

# ============================================================================
# CONFIG OVERLAYS: Colab Training Configs with Drive Sync
# ============================================================================

baseline_colab_config = '''# Baseline Configuration for Colab Training
# Full training with optional Drive sync

experiment:
  name: "baseline_colab"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/baseline"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: false
  max_length: 32

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129
  cache_dir: "/root/.cache/huggingface/datasets"

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  device: "auto"
  save_checkpoints: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

runtime:
  execution_profile: "colab_train"
  sync_to_drive: false  # Set to true or use --sync_to_drive flag
  drive_mount_path: "/content/drive/MyDrive/VLM_Thesis_Outputs"

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/baseline_colab.yaml", 'w') as f:
    f.write(baseline_colab_config)
print("✅ Created: configs/baseline_colab.yaml")

proposed_colab_config = '''# Proposed Model Configuration for Colab Training
# Full training with optional Drive sync

experiment:
  name: "proposed_colab"
  seed: 42
  output_dir: "/content/VLM_Thesis/outputs/proposed"

model:
  name: "Salesforce/blip2-opt-2.7b"
  freeze_vision: true
  freeze_language: true
  use_scene_reasoning: true
  max_length: 32
  
  scene_reasoning:
    hidden_dim: 768
    num_heads: 8
    num_layers: 2
    dropout: 0.1
    use_spatial_encoding: true
    use_relation_attention: true
    spatial_dim: 64
    max_objects: 100

data:
  dataset_name: "HuggingFaceM4/VQAv2"
  fallback_dataset: "Graphcore/vqa"
  train_split: "train[:5000]"
  val_split: "validation[:1000]"
  max_answer_length: 10
  top_k_answers: 3129
  cache_dir: "/root/.cache/huggingface/datasets"

training:
  batch_size: 1
  gradient_accumulation_steps: 8
  learning_rate: 5.0e-5
  weight_decay: 0.01
  num_epochs: 3
  warmup_ratio: 0.1
  max_grad_norm: 1.0
  fp16: true
  device: "auto"
  save_checkpoints: true
  scheduler: "cosine"
  save_steps: 500
  eval_steps: 500
  logging_steps: 50

runtime:
  execution_profile: "colab_train"
  sync_to_drive: false
  drive_mount_path: "/content/drive/MyDrive/VLM_Thesis_Outputs"

evaluation:
  batch_size: 4
  num_beams: 3
  max_length: 32
'''

with open("/content/VLM_Thesis/configs/proposed_colab.yaml", 'w') as f:
    f.write(proposed_colab_config)
print("✅ Created: configs/proposed_colab.yaml")

print("\n" + "="*60)
print("📋 Configuration files summary:")
print("="*60)
print("  ORIGINAL CONFIGS (unchanged):")
print("    baseline.yaml         - BLIP-2 baseline")
print("    proposed.yaml         - BLIP-2 + Scene Reasoning")
print("    ablation_*.yaml       - Ablation study configs")
print("")
print("  MAC DEVELOPMENT OVERLAYS (NEW):")
print("    baseline_mac.yaml     - Dev config for Mac (fp16=off, mps, limited steps)")
print("    proposed_mac.yaml     - Dev config for Mac with Scene Reasoning")
print("")
print("  COLAB TRAINING OVERLAYS (NEW):")
print("    baseline_colab.yaml   - Colab config with Drive sync option")
print("    proposed_colab.yaml   - Colab config with Drive sync option")
print("="*60)

## Section 19: Thesis Assets

Pre-formatted assets for thesis writing: architecture diagrams, tables, and result templates.

In [ ]:
# ============================================================================
# THESIS ASSETS: Architecture Diagram (Mermaid)
# ============================================================================

architecture_diagram = '''# Model Architecture Diagram

## BLIP-2 + Scene Reasoning Module

```mermaid
graph TB
    subgraph Input
        I[Image] --> VE[Vision Encoder<br/>ViT-G/14]
        Q[Question] --> TE[Text Tokenizer]
    end
    
    subgraph BLIP2["BLIP-2 Backbone"]
        VE --> QF[Q-Former<br/>Cross-Modal Alignment]
        TE --> QF
        QF --> LLM[Language Model<br/>OPT-2.7B]
    end
    
    subgraph SceneReasoning["Scene Reasoning Module"]
        VE --> |Visual Features| SP[Spatial Position<br/>Encoding]
        SP --> RA[Relation-Aware<br/>Self-Attention]
        RA --> |Scene Context| FC[Feature<br/>Concatenation]
    end
    
    QF --> FC
    FC --> LLM
    LLM --> A[Answer Generation]
    
    style SceneReasoning fill:#e1f5fe
    style BLIP2 fill:#fff3e0
```

## Detailed Scene Reasoning Module

```mermaid
graph LR
    subgraph SpatialEncoding["Spatial Position Encoding"]
        VF[Visual Features<br/>N×D] --> PE[Sinusoidal<br/>Position Encoding]
        PE --> |+| SF[Spatially-Enhanced<br/>Features]
    end
    
    subgraph RelationAttention["Relation-Aware Attention"]
        SF --> Q2[Query]
        SF --> K[Key]
        SF --> V[Value]
        Q2 --> |scaled dot-product| ATT[Multi-Head<br/>Attention]
        K --> ATT
        V --> ATT
        ATT --> LN[LayerNorm]
        LN --> FFN[Feed-Forward<br/>Network]
        FFN --> OUT[Scene-Aware<br/>Features]
    end
    
    style SpatialEncoding fill:#c8e6c9
    style RelationAttention fill:#bbdefb
```

## Data Flow

```
Input Image (224×224×3)
       ↓
Vision Encoder (ViT-G/14)
       ↓
Visual Features (577×1408)  ──────────────────┐
       ↓                                       ↓
Q-Former Queries (32×768)              Scene Reasoning Module
       ↓                                       ↓
Cross-Attention                        Spatial Encoding
       ↓                                       ↓
Query Embeddings (32×768)              Relation Attention
       ↓                                       ↓
       └───────────── Concatenate ────────────┘
                         ↓
                Language Model (OPT-2.7B)
                         ↓
                  Answer Tokens
```
'''

with open("/content/VLM_Thesis/thesis_assets/architecture_diagram.md", 'w') as f:
    f.write(architecture_diagram)
print("✅ Created: thesis_assets/architecture_diagram.md")

# ============================================================================
# THESIS ASSETS: Experiment Table Template
# ============================================================================

experiment_table = '''# Experimental Results

## Main Comparison

| Model | Exact Match | Normalized Match | Params (M) | Inference (ms) |
|-------|-------------|------------------|------------|----------------|
| BLIP-2 Baseline | 0.0000 | 0.0000 | 3,000 | -- |
| + Scene Reasoning | 0.0000 | 0.0000 | 3,012 | -- |

## Ablation Study

| Configuration | Spatial | Relation | Exact Match | Δ vs Full |
|---------------|---------|----------|-------------|-----------|
| Full Model | ✓ | ✓ | 0.0000 | -- |
| No Spatial | ✗ | ✓ | 0.0000 | -0.00 |
| No Relation | ✓ | ✗ | 0.0000 | -0.00 |
| No Reasoning | ✗ | ✗ | 0.0000 | -0.00 |

## Question Type Analysis

| Question Type | Baseline | Proposed | Improvement |
|---------------|----------|----------|-------------|
| Yes/No | 0.00 | 0.00 | +0.00 |
| Number | 0.00 | 0.00 | +0.00 |
| What | 0.00 | 0.00 | +0.00 |
| Where | 0.00 | 0.00 | +0.00 |
| How | 0.00 | 0.00 | +0.00 |
| Other | 0.00 | 0.00 | +0.00 |

## Training Details

| Hyperparameter | Value |
|----------------|-------|
| Batch Size | 1 |
| Gradient Accumulation | 8 |
| Effective Batch Size | 8 |
| Learning Rate | 5e-5 |
| Weight Decay | 0.01 |
| Warmup Ratio | 0.1 |
| Scheduler | Cosine |
| Epochs | 3 |
| Precision | FP16 |

---
*Table will be updated after running experiments.*
'''

with open("/content/VLM_Thesis/thesis_assets/experiment_table.md", 'w') as f:
    f.write(experiment_table)
print("✅ Created: thesis_assets/experiment_table.md")

# ============================================================================
# THESIS ASSETS: Results Summary Template
# ============================================================================

results_summary = '''# Results Summary

## Executive Summary

This work presents a Scene Reasoning Module that enhances BLIP-2's visual 
question answering capabilities through explicit spatial relationship modeling.

### Key Contributions

1. **Scene Reasoning Module**: A lightweight attention-based module that 
   captures spatial relationships between visual elements.

2. **Spatial Position Encoding**: Sinusoidal encoding of 2D positions 
   enables the model to reason about object locations.

3. **Relation-Aware Attention**: Multi-head self-attention mechanism 
   that models pairwise object relationships.

### Main Results

- **Baseline (BLIP-2)**: [X.XX]% accuracy on VQAv2 validation
- **Proposed Model**: [Y.YY]% accuracy on VQAv2 validation  
- **Improvement**: +[Z.ZZ]% absolute improvement

### Ablation Findings

| Component | Contribution |
|-----------|-------------|
| Spatial Encoding | +X.XX% |
| Relation Attention | +Y.YY% |
| Combined | +Z.ZZ% |

### Qualitative Analysis

The Scene Reasoning Module shows particular strength in:
- Questions requiring spatial reasoning ("Where is...", "What is next to...")
- Counting questions where object relationships matter
- Scene understanding questions

### Limitations

1. Increased inference time due to additional attention layers
2. Limited improvement on simple yes/no questions
3. Memory constraints on larger batch sizes

### Future Work

1. Integration with object detection for explicit object-level reasoning
2. Extension to video question answering
3. Multi-scale scene reasoning

---
*This summary will be updated with actual experimental results.*
'''

with open("/content/VLM_Thesis/thesis_assets/results_summary.md", 'w') as f:
    f.write(results_summary)
print("✅ Created: thesis_assets/results_summary.md")

# ============================================================================
# THESIS ASSETS: LaTeX Table Template
# ============================================================================

latex_tables = r'''% LaTeX Tables for Thesis

% Main Results Table
\begin{table}[h]
\centering
\caption{Main experimental results on VQAv2 validation set.}
\label{tab:main_results}
\begin{tabular}{lcccc}
\toprule
\textbf{Model} & \textbf{Exact} & \textbf{Normalized} & \textbf{Params} & \textbf{Time} \\
\midrule
BLIP-2 Baseline & 0.00 & 0.00 & 3.0B & -- \\
+ Scene Reasoning & 0.00 & 0.00 & 3.01B & -- \\
\bottomrule
\end{tabular}
\end{table}

% Ablation Study Table
\begin{table}[h]
\centering
\caption{Ablation study of Scene Reasoning Module components.}
\label{tab:ablation}
\begin{tabular}{lccc}
\toprule
\textbf{Configuration} & \textbf{Spatial} & \textbf{Relation} & \textbf{Accuracy} \\
\midrule
Full Model & \checkmark & \checkmark & 0.00 \\
No Spatial & $\times$ & \checkmark & 0.00 \\
No Relation & \checkmark & $\times$ & 0.00 \\
No Reasoning & $\times$ & $\times$ & 0.00 \\
\bottomrule
\end{tabular}
\end{table}

% Question Type Analysis
\begin{table}[h]
\centering
\caption{Performance breakdown by question type.}
\label{tab:question_types}
\begin{tabular}{lccc}
\toprule
\textbf{Type} & \textbf{Baseline} & \textbf{Proposed} & \textbf{$\Delta$} \\
\midrule
Yes/No & 0.00 & 0.00 & +0.00 \\
Number & 0.00 & 0.00 & +0.00 \\
What & 0.00 & 0.00 & +0.00 \\
Where & 0.00 & 0.00 & +0.00 \\
\bottomrule
\end{tabular}
\end{table}
'''

with open("/content/VLM_Thesis/thesis_assets/latex_tables.tex", 'w') as f:
    f.write(latex_tables)
print("✅ Created: thesis_assets/latex_tables.tex")

print("\n📚 Thesis assets created successfully!")

## Section 20: Smoke Test

Verify all components work before running full training.

In [ ]:
# ============================================================================
# SMOKE TEST: Verify all imports work
# ============================================================================

print("🔍 Running import verification...")
print("="*60)

try:
    # Core utilities
    from src.utils.config import load_config
    from src.utils.seed import set_seed
    from src.utils.io import save_checkpoint, load_checkpoint
    from src.utils.logging import TensorBoardLogger
    print("✅ Utils imports successful")
except Exception as e:
    print(f"❌ Utils import failed: {e}")

try:
    # Dataset
    from src.datasets.vqa_dataset import VQADataset, create_dataloaders
    from src.datasets.answer_vocab import AnswerVocabulary
    print("✅ Dataset imports successful")
except Exception as e:
    print(f"❌ Dataset import failed: {e}")

try:
    # Models
    from src.models.blip2_wrapper import BLIP2Wrapper
    from src.models.vqa_head import VQAHead
    from src.models.scene_reasoning import SceneReasoningModule, SceneReasoningConfig
    print("✅ Model imports successful")
except Exception as e:
    print(f"❌ Model import failed: {e}")

try:
    # Training
    from src.training.losses import LabelSmoothingCrossEntropy
    from src.training.metrics import VQAEvaluator
    from src.training.schedulers import get_scheduler
    from src.training.trainer import VQATrainer
    print("✅ Training imports successful")
except Exception as e:
    print(f"❌ Training import failed: {e}")

try:
    # Evaluation
    from src.evaluation.evaluate import evaluate_model
    from src.evaluation.error_analysis import ErrorAnalyzer
    print("✅ Evaluation imports successful")
except Exception as e:
    print(f"❌ Evaluation import failed: {e}")

print("="*60)
print("🎉 All imports verified!")

In [ ]:
# ============================================================================
# SMOKE TEST: Test Scene Reasoning Module
# ============================================================================

import torch
from src.models.scene_reasoning import SceneReasoningModule, SceneReasoningConfig

print("🔍 Testing Scene Reasoning Module...")
print("="*60)

# Create config
config = SceneReasoningConfig(
    hidden_dim=768,
    num_heads=8,
    num_layers=2,
    use_spatial_encoding=True,
    use_relation_attention=True,
)

# Create module
module = SceneReasoningModule(config)
print(f"✅ Created SceneReasoningModule")
print(f"   Parameters: {sum(p.numel() for p in module.parameters()):,}")

# Test forward pass
batch_size = 2
seq_len = 100
hidden_dim = 768

dummy_features = torch.randn(batch_size, seq_len, hidden_dim)
dummy_positions = torch.rand(batch_size, seq_len, 4)  # [x, y, w, h]

output, attention_weights = module(dummy_features, dummy_positions)

print(f"✅ Forward pass successful")
print(f"   Input shape: {dummy_features.shape}")
print(f"   Output shape: {output.shape}")
print(f"   Attention weights: {len(attention_weights)} layers")

# Verify shapes
assert output.shape == dummy_features.shape, "Output shape mismatch!"
print("✅ Shape verification passed")

print("="*60)
print("🎉 Scene Reasoning Module working correctly!")

In [ ]:
# ============================================================================
# SMOKE TEST: Load and test BLIP-2 model (Colab GPU required)
# ============================================================================

import torch
from transformers import Blip2Processor

print("🔍 Testing BLIP-2 Wrapper...")
print("="*60)

# Check GPU availability
if torch.cuda.is_available():
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    print("⚠️  No GPU available - will run on CPU (slow)")

# Test processor loading
print("\n📥 Loading BLIP-2 processor...")
processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-2.7b")
print("✅ Processor loaded")

# Test with dummy image
from PIL import Image
import numpy as np

dummy_image = Image.fromarray(np.random.randint(0, 255, (224, 224, 3), dtype=np.uint8))
dummy_question = "What is in this image?"

inputs = processor(images=dummy_image, text=dummy_question, return_tensors="pt")
print(f"✅ Processor test passed")
print(f"   pixel_values: {inputs['pixel_values'].shape}")
print(f"   input_ids: {inputs['input_ids'].shape}")

print("="*60)
print("🎉 BLIP-2 processor ready!")
print("\n⚠️  Note: Full model loading will happen during training.")

In [ ]:
# ============================================================================
# SMOKE TEST: Test dataset loading (small sample)
# ============================================================================

from datasets import load_dataset

print("🔍 Testing dataset loading...")
print("="*60)

try:
    # Try primary dataset
    print("📥 Attempting to load HuggingFaceM4/VQAv2...")
    dataset = load_dataset("HuggingFaceM4/VQAv2", split="validation[:10]")
    print(f"✅ Loaded VQAv2: {len(dataset)} samples")
    print(f"   Columns: {dataset.column_names}")
except Exception as e:
    print(f"⚠️  VQAv2 failed: {e}")
    print("📥 Trying fallback dataset...")
    try:
        dataset = load_dataset("Graphcore/vqa", split="validation[:10]")
        print(f"✅ Loaded Graphcore/vqa: {len(dataset)} samples")
        print(f"   Columns: {dataset.column_names}")
    except Exception as e2:
        print(f"❌ Fallback also failed: {e2}")
        dataset = None

if dataset:
    print("\n📋 Sample data:")
    sample = dataset[0]
    for key, value in sample.items():
        if key == 'image':
            print(f"   {key}: PIL Image {value.size}")
        elif isinstance(value, str) and len(value) > 50:
            print(f"   {key}: {value[:50]}...")
        else:
            print(f"   {key}: {value}")

print("="*60)
print("🎉 Dataset loading verified!")

## Section 21: Run Experiments

Commands to train baseline and proposed models, run ablations, and generate reports.

In [ ]:
# ============================================================================
# TRAINING: Baseline Model (BLIP-2 without Scene Reasoning)
# ============================================================================
# Uncomment and run when ready to train

print("📋 Baseline Training Command:")
print("="*60)
print("!python scripts/train.py --config configs/baseline.yaml")
print("="*60)

# Uncomment the line below to run training:
# !cd /content/VLM_Thesis && python scripts/train.py --config configs/baseline.yaml

In [ ]:
# ============================================================================
# TRAINING: Proposed Model (BLIP-2 + Scene Reasoning)
# ============================================================================
# Uncomment and run when ready to train

print("📋 Proposed Model Training Command:")
print("="*60)
print("!python scripts/train.py --config configs/proposed.yaml")
print("="*60)

# Uncomment the line below to run training:
# !cd /content/VLM_Thesis && python scripts/train.py --config configs/proposed.yaml

In [ ]:
# ============================================================================
# ABLATION STUDIES: Run all ablation experiments
# ============================================================================
# Uncomment and run when ready to train

print("📋 Ablation Study Commands:")
print("="*60)
print("1. No Spatial Encoding:")
print("   !python scripts/train.py --config configs/ablation_no_spatial.yaml")
print()
print("2. No Relation Attention:")
print("   !python scripts/train.py --config configs/ablation_no_relation.yaml")
print()
print("3. No Scene Reasoning (baseline):")
print("   !python scripts/train.py --config configs/ablation_no_reasoning.yaml")
print("="*60)

# Uncomment to run ablations:
# !cd /content/VLM_Thesis && python scripts/train.py --config configs/ablation_no_spatial.yaml
# !cd /content/VLM_Thesis && python scripts/train.py --config configs/ablation_no_relation.yaml
# !cd /content/VLM_Thesis && python scripts/train.py --config configs/ablation_no_reasoning.yaml

In [ ]:
# ============================================================================
# EVALUATION: Evaluate trained models
# ============================================================================

print("📋 Evaluation Commands:")
print("="*60)
print("Baseline Evaluation:")
print("   !python scripts/eval.py --checkpoint outputs/baseline/best_model.pt --config configs/baseline.yaml")
print()
print("Proposed Model Evaluation:")
print("   !python scripts/eval.py --checkpoint outputs/proposed/best_model.pt --config configs/proposed.yaml")
print("="*60)

# Uncomment to run evaluation:
# !cd /content/VLM_Thesis && python scripts/eval.py --checkpoint outputs/baseline/best_model.pt --config configs/baseline.yaml
# !cd /content/VLM_Thesis && python scripts/eval.py --checkpoint outputs/proposed/best_model.pt --config configs/proposed.yaml

In [ ]:
# ============================================================================
# REPORT GENERATION: Generate thesis-ready reports
# ============================================================================

print("📋 Report Generation Command:")
print("="*60)
print("!python scripts/make_report.py --results_dir outputs/ --output_dir thesis_assets/")
print("="*60)

# Uncomment to generate reports after experiments:
# !cd /content/VLM_Thesis && python scripts/make_report.py --results_dir outputs/ --output_dir thesis_assets/

## Section 22: TensorBoard Visualization

Monitor training progress with TensorBoard.

In [ ]:
# ============================================================================
# TENSORBOARD: Launch TensorBoard for training monitoring
# ============================================================================

# Load TensorBoard extension
%load_ext tensorboard

# Launch TensorBoard
# %tensorboard --logdir /content/VLM_Thesis/outputs

print("📊 TensorBoard Setup:")
print("="*60)
print("Uncomment the line above to launch TensorBoard")
print("Log directories will be under: /content/VLM_Thesis/outputs/*/logs")
print("="*60)

## Section 23: Project README

Create project documentation.

In [ ]:
# ============================================================================
# PROJECT README
# ============================================================================

readme_content = '''# Vision-Language Models for Scene Understanding and VQA

## Overview

This project implements a Scene Reasoning Module that enhances BLIP-2's visual 
question answering capabilities through explicit spatial relationship modeling.

**Research Question**: Can explicit scene structure modeling improve VQA performance
on questions requiring spatial reasoning?

## Architecture

```
BLIP-2 (Baseline)
├── Vision Encoder (ViT-G/14, frozen)
├── Q-Former (cross-modal alignment)
└── Language Model (OPT-2.7B, frozen)

+ Scene Reasoning Module (Proposed)
  ├── Spatial Position Encoding (sinusoidal 2D)
  └── Relation-Aware Self-Attention (multi-head)
```

## Project Structure

```
VLM_Thesis/
├── configs/                    # YAML configuration files
│   ├── baseline.yaml          # BLIP-2 baseline
│   ├── proposed.yaml          # BLIP-2 + Scene Reasoning
│   └── ablation_*.yaml        # Ablation study configs
├── src/
│   ├── datasets/              # Dataset loaders
│   ├── models/                # Model implementations
│   ├── training/              # Training utilities
│   ├── evaluation/            # Evaluation and analysis
│   └── utils/                 # Helper utilities
├── scripts/                   # Entry point scripts
│   ├── train.py              # Training script
│   ├── eval.py               # Evaluation script
│   └── make_report.py        # Report generation
├── outputs/                   # Experiment outputs
└── thesis_assets/             # Thesis-ready materials
```

## Quick Start (Google Colab)

### 1. Setup Environment
```python
# Run all cells in Sections 1-19 to set up the project
```

### 2. Run Smoke Tests
```python
# Run Section 20 cells to verify installation
```

### 3. Train Baseline
```bash
!python scripts/train.py --config configs/baseline.yaml
```

### 4. Train Proposed Model
```bash
!python scripts/train.py --config configs/proposed.yaml
```

### 5. Run Ablations
```bash
!python scripts/train.py --config configs/ablation_no_spatial.yaml
!python scripts/train.py --config configs/ablation_no_relation.yaml
```

### 6. Evaluate
```bash
!python scripts/eval.py --checkpoint outputs/proposed/best_model.pt --config configs/proposed.yaml
```

### 7. Generate Reports
```bash
!python scripts/make_report.py --results_dir outputs/
```

## Key Components

### Scene Reasoning Module

The core contribution is a lightweight attention-based module:

- **Spatial Position Encoding**: Sinusoidal encoding of 2D positions
- **Relation-Aware Attention**: Multi-head self-attention for object relationships

```python
from src.models.scene_reasoning import SceneReasoningModule, SceneReasoningConfig

config = SceneReasoningConfig(
    hidden_dim=768,
    num_heads=8,
    num_layers=2,
    use_spatial_encoding=True,
    use_relation_attention=True,
)
module = SceneReasoningModule(config)
```

### Configuration System

All experiments are controlled via YAML configs:

```yaml
model:
  use_scene_reasoning: true
  scene_reasoning:
    hidden_dim: 768
    use_spatial_encoding: true
    use_relation_attention: true
```

## Experiments

| Experiment | Config File | Description |
|------------|-------------|-------------|
| Baseline | `baseline.yaml` | BLIP-2 without scene reasoning |
| Proposed | `proposed.yaml` | Full model with all components |
| Ablation 1 | `ablation_no_spatial.yaml` | Without spatial encoding |
| Ablation 2 | `ablation_no_relation.yaml` | Without relation attention |

## Hardware Requirements

- **Minimum**: Google Colab Free (T4 GPU, 16GB RAM)
- **Recommended**: Google Colab Pro (A100 GPU, High-RAM)

Memory-saving features:
- Frozen BLIP-2 backbone
- FP16 mixed precision
- Gradient accumulation (effective batch size = 8)

## Dataset

Using VQAv2 dataset via HuggingFace:
- Primary: `HuggingFaceM4/VQAv2`
- Fallback: `Graphcore/vqa`

## Citation

If you use this code, please cite:

```bibtex
@thesis{vlm_scene_reasoning_2024,
  title={Vision-Language Models for Scene Understanding and VQA},
  author={[Your Name]},
  year={2024},
  school={[Your Institution]}
}
```

## License

MIT License
'''

with open("/content/VLM_Thesis/README.md", 'w') as f:
    f.write(readme_content)
print("✅ Created: README.md")

## Section 24: Requirements File

Create requirements.txt for reproducibility.

In [ ]:
# ============================================================================
# REQUIREMENTS FILE
# ============================================================================

requirements_content = '''# Vision-Language VQA Research Project
# Requirements for Google Colab environment

# Core ML frameworks
torch>=2.0.0
torchvision>=0.15.0

# HuggingFace ecosystem
transformers>=4.35.0
datasets>=2.14.0
accelerate>=0.24.0

# Image processing
Pillow>=9.0.0

# Utilities
numpy>=1.21.0
tqdm>=4.65.0
pyyaml>=6.0
tensorboard>=2.14.0

# Optional: Weights & Biases
# wandb>=0.15.0

# Development
pytest>=7.0.0
black>=23.0.0
'''

with open("/content/VLM_Thesis/requirements.txt", 'w') as f:
    f.write(requirements_content)
print("✅ Created: requirements.txt")

## Section 25: Git Configuration and .gitignore

Configure Git for the Mac-first + Colab-train workflow.

In [ ]:
# ============================================================================
# GITIGNORE: Prevent uploading outputs, cache, and checkpoints
# ============================================================================

gitignore_content = '''# =============================================================================
# VLM Thesis Project - .gitignore
# =============================================================================
# This file ensures only SOURCE CODE is synced between Mac and Colab.
# Outputs, checkpoints, and cache are NOT uploaded to avoid large repo size.
# =============================================================================

# -----------------------------------------------------------------------------
# Training Outputs (NEVER upload - generated on Colab)
# -----------------------------------------------------------------------------
outputs/
thesis_assets/*.csv
thesis_assets/experiment_*.md
thesis_assets/results_summary.md

# Keep templates
!thesis_assets/architecture_diagram.md
!thesis_assets/latex_tables.tex

# -----------------------------------------------------------------------------
# Model Checkpoints (NEVER upload - too large)
# -----------------------------------------------------------------------------
*.pt
*.pth
*.bin
*.safetensors
checkpoints/

# -----------------------------------------------------------------------------
# Cache Directories
# -----------------------------------------------------------------------------
__pycache__/
*.py[cod]
*$py.class
.pytest_cache/
.mypy_cache/

# HuggingFace cache (download fresh on each env)
.cache/
~/.cache/huggingface/

# Dataset cache
*.arrow
*.lock

# -----------------------------------------------------------------------------
# Environment and IDE
# -----------------------------------------------------------------------------
.env
.venv/
venv/
ENV/

# VS Code
.vscode/
*.code-workspace

# Jupyter/Colab
.ipynb_checkpoints/
*.ipynb_checkpoints/

# macOS
.DS_Store
.AppleDouble
.LSOverride

# -----------------------------------------------------------------------------
# Logs and Temporary Files
# -----------------------------------------------------------------------------
logs/
*.log
wandb/
runs/
tensorboard_logs/

# Temporary files
*.tmp
*.temp
*.swp
*~

# -----------------------------------------------------------------------------
# Large Data Files (download separately)
# -----------------------------------------------------------------------------
data/
*.zip
*.tar.gz
*.tar
*.h5
*.hdf5

# -----------------------------------------------------------------------------
# WHAT TO COMMIT (for reference)
# -----------------------------------------------------------------------------
# ✅ src/**/*.py           - All source code
# ✅ scripts/*.py          - Training/eval scripts
# ✅ configs/*.yaml        - All config files (including overlays)
# ✅ .github/              - GitHub config and copilot instructions
# ✅ README.md             - Project documentation
# ✅ requirements.txt      - Dependencies
# ✅ VLM_VQA_Research.ipynb - Main notebook
# ✅ thesis_assets/architecture_diagram.md - Architecture diagrams
# ✅ thesis_assets/latex_tables.tex - LaTeX templates
# -----------------------------------------------------------------------------
'''

with open("/content/VLM_Thesis/.gitignore", 'w') as f:
    f.write(gitignore_content)
print("✅ Created: .gitignore")

print("\n📋 Files that WILL be tracked by Git:")
print("   ✅ src/**/*.py, scripts/*.py, configs/*.yaml")
print("   ✅ .github/, README.md, requirements.txt")
print("   ✅ VLM_VQA_Research.ipynb")
print("\n📋 Files that will NOT be tracked (too large/generated):")
print("   ❌ outputs/, checkpoints/, *.pt, *.pth")
print("   ❌ __pycache__/, .cache/, .ipynb_checkpoints/")

## Section 26: Git-Based Sync Workflow (VS Code ⇄ Colab)

Documentation for the Mac-first + Colab-train development workflow.

In [ ]:
# ============================================================================
# GIT-BASED SYNC WORKFLOW DOCUMENTATION
# ============================================================================

workflow_doc = '''# Mac-First + Colab-Train Workflow

## Overview

This project uses a **Git-based sync workflow** where:
- **Mac (VS Code)**: Primary development environment for writing/editing code
- **Colab GPU**: Exclusive environment for training and heavy computation
- **Git**: Sync mechanism between environments

```
┌─────────────────┐                      ┌─────────────────┐
│                 │      git push        │                 │
│   Mac / VS Code │ ──────────────────▶  │     GitHub      │
│   (Development) │                      │   (Repository)  │
│                 │ ◀──────────────────  │                 │
└─────────────────┘      git pull        └─────────────────┘
                                                │
                                                │ git clone / pull
                                                ▼
                                         ┌─────────────────┐
                                         │                 │
                                         │   Google Colab  │
                                         │   (Training)    │
                                         │                 │
                                         └─────────────────┘
```

## Execution Profiles

| Profile | Environment | Purpose | Training Allowed |
|---------|-------------|---------|------------------|
| `mac_dev` | Mac/Local | Development, testing | Smoke tests only (≤10 steps) |
| `colab_train` | Colab GPU | Full training | Yes (unlimited) |
| `eval_only` | Any | Evaluation only | No |

## Step-by-Step Workflow

### 1. Initial Setup (One-time)

**On Mac:**
```bash
# Clone repository
git clone https://github.com/YOUR_USERNAME/VLM_Thesis.git
cd VLM_Thesis

# Open in VS Code
code .
```

**On Colab:**
```python
# Clone repository in Colab
!git clone https://github.com/YOUR_USERNAME/VLM_Thesis.git
%cd /content/VLM_Thesis

# Run all notebook cells to create project structure
```

### 2. Development Cycle

#### A. Write/Edit Code (Mac)
```bash
# Edit code in VS Code
# Test locally with mac_dev profile
python scripts/train.py --config configs/baseline_mac.yaml --execution_profile mac_dev

# Commit and push
git add .
git commit -m "Updated scene reasoning module"
git push origin main
```

#### B. Train on Colab
```python
# Pull latest changes
!cd /content/VLM_Thesis && git pull origin main

# Run full training
!python scripts/train.py --config configs/proposed_colab.yaml --sync_to_drive
```

#### C. Get Results Back (Optional)
```python
# Option 1: Use Drive sync (automatic if --sync_to_drive used)
# Results saved to: /content/drive/MyDrive/VLM_Thesis_Outputs/

# Option 2: Download manually
from google.colab import files
!zip -r results.zip outputs/
files.download('results.zip')
```

### 3. Config Files by Environment

**For Mac Development:**
- `configs/baseline_mac.yaml` - Baseline with Mac-safe settings
- `configs/proposed_mac.yaml` - Proposed model with Mac-safe settings

**For Colab Training:**
- `configs/baseline_colab.yaml` - Baseline with Drive sync option
- `configs/proposed_colab.yaml` - Proposed with Drive sync option

**Original Configs (work on both):**
- `configs/baseline.yaml` - Uses auto-detection
- `configs/proposed.yaml` - Uses auto-detection

## Safety Guards

### Mac Development (`mac_dev` profile)
```
⚠️ The following limits are enforced:
   - max_steps: 10 (cannot exceed)
   - max_samples: 50 (cannot exceed)
   - fp16: disabled (MPS limitation)
   - checkpoints: disabled by default
   
Attempting to exceed these limits will raise an error.
Use Colab for full training.
```

### Preventing Accidental Long Training
```python
# This will FAIL on Mac:
python scripts/train.py --config configs/proposed.yaml --epochs 10

# Error: 🛑 SAFETY GUARD: mac_dev profile does not allow...

# This is the correct way on Mac:
python scripts/train.py --config configs/proposed_mac.yaml
```

## Important Rules

1. **NEVER use Google Drive as code source**
   - Git is the single source of truth
   - Drive is for OUTPUT sync only

2. **NEVER commit outputs or checkpoints**
   - .gitignore prevents this automatically
   - Outputs are generated fresh on Colab

3. **NEVER modify model size/architecture for Mac**
   - Mac configs only change runtime settings
   - Model (BLIP-2 + Scene Reasoning) is identical

4. **ALWAYS test locally before pushing**
   ```bash
   python scripts/train.py --config configs/baseline_mac.yaml --smoke_test true
   ```

## Troubleshooting

### "Module not found" on Colab
```python
# Re-run the notebook cells to recreate project structure
# OR manually add to path:
import sys
sys.path.insert(0, '/content/VLM_Thesis')
```

### Git conflicts
```bash
# On Mac - force pull (discard local changes to outputs)
git fetch origin
git reset --hard origin/main
```

### Colab timeout during training
```python
# Use Drive sync to preserve progress
!python scripts/train.py --config configs/proposed_colab.yaml --sync_to_drive

# Checkpoints are saved to Drive automatically
```
'''

with open("/content/VLM_Thesis/docs/WORKFLOW.md", 'w') as f:
    os.makedirs("/content/VLM_Thesis/docs", exist_ok=True)
    f.write(workflow_doc)
print("✅ Created: docs/WORKFLOW.md")

# Print summary
print("\n" + "="*70)
print("📘 MAC-FIRST + COLAB-TRAIN WORKFLOW")
print("="*70)
print("""
DEVELOPMENT (Mac/VS Code):
  1. Edit code in VS Code
  2. Test: python scripts/train.py --config configs/baseline_mac.yaml
  3. Commit: git add . && git commit -m "message" && git push

TRAINING (Colab):
  1. Pull: !git pull origin main
  2. Train: !python scripts/train.py --config configs/proposed_colab.yaml
  3. Sync: Add --sync_to_drive to save results to Google Drive

SAFETY:
  - mac_dev profile limits: max 10 steps, max 50 samples
  - Full training ONLY on Colab with colab_train profile
  - Model architecture is UNCHANGED across environments
""")
print("="*70)

## Section 25: Final Project Summary

Summary of the complete project structure and next steps.

In [ ]:
# ============================================================================
# FINAL PROJECT SUMMARY
# ============================================================================

import os

def count_files(directory):
    """Count Python and YAML files in directory."""
    py_count = 0
    yaml_count = 0
    for root, dirs, files in os.walk(directory):
        for f in files:
            if f.endswith('.py'):
                py_count += 1
            elif f.endswith('.yaml'):
                yaml_count += 1
    return py_count, yaml_count

print("="*70)
print("🎉 VLM THESIS PROJECT - COMPLETE SETUP SUMMARY")
print("="*70)

print("\n📁 PROJECT STRUCTURE:")
print("-"*70)

# List all directories
for item in sorted(os.listdir("/content/VLM_Thesis")):
    path = os.path.join("/content/VLM_Thesis", item)
    if os.path.isdir(path):
        py, yaml = count_files(path)
        print(f"   📂 {item}/")
        if py > 0:
            print(f"      └── {py} Python files")
        if yaml > 0:
            print(f"      └── {yaml} YAML configs")
    else:
        print(f"   📄 {item}")

print("\n📊 MODULE SUMMARY:")
print("-"*70)
print("   src/utils/     - Configuration, logging, checkpointing, seeding")
print("   src/data/      - VQA dataset loader, answer vocabulary")
print("   src/models/    - BLIP-2 wrapper, VQA head, Scene Reasoning Module")
print("   src/training/  - Trainer, losses, metrics, schedulers")
print("   src/evaluation/- Evaluation pipeline, error analysis")
print("   scripts/       - train.py, eval.py, make_report.py")
print("   configs/       - Experiment configurations (5 configs)")

print("\n🔬 EXPERIMENTS READY:")
print("-"*70)
print("   1. Baseline    - BLIP-2 without scene reasoning")
print("   2. Proposed    - BLIP-2 + Scene Reasoning Module")
print("   3. Ablation 1  - Without spatial encoding")
print("   4. Ablation 2  - Without relation attention")
print("   5. Ablation 3  - Without any scene reasoning")

print("\n📝 THESIS ASSETS:")
print("-"*70)
print("   - Architecture diagram (Mermaid)")
print("   - Experiment results table template")
print("   - Results summary template")
print("   - LaTeX table templates")

print("\n🚀 NEXT STEPS:")
print("-"*70)
print("   1. Run all cells (Sections 1-19) to create project files")
print("   2. Run smoke tests (Section 20) to verify setup")
print("   3. Train baseline: !python scripts/train.py --config configs/baseline.yaml")
print("   4. Train proposed: !python scripts/train.py --config configs/proposed.yaml")
print("   5. Run ablations")
print("   6. Generate reports: !python scripts/make_report.py")
print("   7. Copy thesis_assets/ content to your thesis document")

print("\n" + "="*70)
print("✅ PROJECT READY FOR EXPERIMENTS!")
print("="*70)

## 🚀 Quick Start for Colab

**If you're running on Colab for the first time:**
1. Run Cell 1 (Colab Setup) to clone repo and install dependencies
2. Run all cells in order (Runtime → Run all)

**To update existing installation:**
```python
!cd /content/VLM_Thesis && git pull
```

**To start training:**
```python
!python scripts/train.py --config configs/proposed_colab.yaml --sync_to_drive
```